In [524]:
#setup Odoo

import requests

jsonrpcid = 0
url = "https://demo-28-08-6.adhoc.ar/jsonrpc"
db = "demo-28-08-6"
username = "Chargebee"
password = "Chargebee"
headers = {'Content-Type': 'application/json'}


class Odoo():

    def __init__(self):
        self.jsonrpcid = 0

    def call(self, method: str, service: str, *args):
        self.jsonrpcid += 1

        data = {
            "jsonrpc": "2.0",
            "method": "call",
            "params": {
                "service": service,
                "method": method,
                "args": args
            },
            "id": self.jsonrpcid
        }

        # Realizar solicitud HTTP POST
        response = requests.post(url, headers=headers, json=data)
        result = response.json()

        return result

    def login(self, db: str, username: str, password: str):
        self.db = db
        self.username = username
        self.password = password
        outcome = self.call('login', 'common', self.db, self.username, self.password)
        if 'result' not in outcome:
            raise Exception('Failed to login')
        self.uid = outcome['result']

    def execute(self, *args):
        result = self.call('execute_kw', 'object', self.db, self.uid, self.password, *args)
        r = result.get('result', False)
        if not r:
            print(result)
            raise Exception('Command has failed')
        return r

    def read(self, model: str, ids: list=[], fields: list=[], context: dict={}):
        return self.execute(model, 'read', [ids, fields], {'context': context})

    def search_read(self, model: str, domain: list=[], fields: list=[], offset: int=0, limit: int=10, context: dict={}):
        return self.execute(model, 'search_read', [domain, fields, offset, limit], {'context': context})

    def create(self, model: str, vals_list: list=[], context: dict={}):
        return self.execute(model, 'create', vals_list, {'context': context})

# Crear nueva instancia
odoo = Odoo()

# Realizar login
odoo.login(db, username, password)

In [2]:
#setup chargebee

import chargebee

chargebee.configure('live_3NEiAmTg7dQArgRHR04LvHaIABupcuQOS', 'cliengo')

In [456]:
# Obtener las invoices desde Chargebee

import pandas as pd
import time
def get_chargebee_invoices_august_2024():
    invoice_list = []
    offset = None  # Inicialmente sin offset para la paginación
    start_date = int(time.mktime(time.strptime("2024-08-16", "%Y-%m-%d")))  # 1 de agosto de 2024 en Unix Timestamp
    end_date = int(time.mktime(time.strptime("2024-08-23 23:59:59", "%Y-%m-%d %H:%M:%S")))  # 31 de agosto de 2024

    while True:
        # Obtener la lista de facturas, sin límite explícito, pero con paginación
        entries = chargebee.Invoice.list({
            "status[in]": ["paid", "payment_due"],
            "date[between]": [start_date, end_date],  # Filtro por rango de fechas en Unix Timestamp
            "sort_by[asc]": "date",
            "limit": 100,  # Procesar 100 facturas por página
            "offset": offset  # Usar el offset actual para paginación
        })

        # Agregar las facturas a la lista
        for entry in entries:
            invoice = entry.invoice
            invoice_list.append(invoice.__dict__)  # Agregar la factura a la lista

        # Verificar si hay más páginas (si existe un offset para la siguiente página)
        offset = entries.next_offset
        if not offset:
            break  # Salir del bucle si no hay más páginas

    return invoice_list

# Llamar la función para obtener todas las facturas de agosto de 2024
invoices = get_chargebee_invoices_august_2024()

In [457]:
# Convertir la lista de diccionarios en un DataFrame de pandas
dfchar = pd.DataFrame(invoices)

# Mostrar el DataFrame
dfchar

,values,sub_types,dependant_types,id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,void_reason_code,deleted,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,None,False,None,ES,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '186757', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186757,Mexico,5be3127ae4b0e9b0e27b3220,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '186758', 'po_number': 'CUENTA TEST', '...",{'line_items': <class 'chargebee.models.invoic...,{},186758,CUENTA TEST,64a812e0a46cbf0032044e4c,16CaLfTvmAKjx1inK,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,"{'id': '187346', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187346,Mexico,663d4a0104d4ea377b73e976,16BiiCUDdoBbY6l7c,True,paid,ROCM700304L95,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
576,"{'id': '187347', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187347,Mexico,6168bafc84869d002a2afe30,16A16fSyJAFdS2oAf,True,paid,BAMF660710BK3,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
577,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
578,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [458]:
#le renombre al campo id para que se identifique bien ya que habran varios campos que se llaman id en el dataframe final
df = dfchar.rename(columns={'id': 'invoice_id'})
df

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,void_reason_code,deleted,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,None,False,None,ES,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '186757', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186757,Mexico,5be3127ae4b0e9b0e27b3220,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '186758', 'po_number': 'CUENTA TEST', '...",{'line_items': <class 'chargebee.models.invoic...,{},186758,CUENTA TEST,64a812e0a46cbf0032044e4c,16CaLfTvmAKjx1inK,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,"{'id': '187346', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187346,Mexico,663d4a0104d4ea377b73e976,16BiiCUDdoBbY6l7c,True,paid,ROCM700304L95,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
576,"{'id': '187347', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187347,Mexico,6168bafc84869d002a2afe30,16A16fSyJAFdS2oAf,True,paid,BAMF660710BK3,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
577,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
578,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [355]:
#para separar los campos dentro de jsons y diccionarios genere el siguiente codigo @Yepes
def print_column_types(dataframe):
    for column in dataframe.columns:
        dtype = dataframe[column].dtype
        print(f"{column} \t {dtype}")

# Llamar a la función para mostrar las columnas y tipos de datos
print_column_types(df)

values 	 object
sub_types 	 object
dependant_types 	 object
invoice_id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
vat_number 	 object
price_type 	 object
date 	 int64
due_date 	 int64
net_term_days 	 int64
exchange_rate 	 float64
currency_code 	 object
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 int64
dunning_status 	 object
next_retry_at 	 object
voided_at 	 object
resource_version 	 int64
updated_at 	 int64
sub_total 	 int64
sub_total_in_local_currency 	 object
total_in_local_currency 	 object
local_currency_code 	 object
tax 	 int64
local_currency_exchange_rate 	 object
first_invoice 	 bool
new_sales_amount 	 float64
has_advance_charges 	 bool
term_finalized 	 bool
is_gifted 	 bool
generated_at 	 int64
expected_payment_date 	 float64
amount_to_collect 	 int64
round_off_amount 	 int64
line_items 	 object
discounts 	 object
line_it

In [459]:
df_objects = df.select_dtypes(include=['object'])
df_objects

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,status,vat_number,price_type,...,payment_owner,void_reason_code,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,paid,30680124104,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,paid,DPL060130RE6,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,paid,A58075250,tax_exclusive,...,None,None,None,ES,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '186757', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186757,Mexico,5be3127ae4b0e9b0e27b3220,1mkVvw2RCFlS8NxFf,paid,CET0810145K6,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '186758', 'po_number': 'CUENTA TEST', '...",{'line_items': <class 'chargebee.models.invoic...,{},186758,CUENTA TEST,64a812e0a46cbf0032044e4c,16CaLfTvmAKjx1inK,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,"{'id': '187346', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187346,Mexico,663d4a0104d4ea377b73e976,16BiiCUDdoBbY6l7c,paid,ROCM700304L95,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
576,"{'id': '187347', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187347,Mexico,6168bafc84869d002a2afe30,16A16fSyJAFdS2oAf,paid,BAMF660710BK3,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
577,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,paid,SSB120314V52,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
578,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,paid,110860111,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [1]:
# List of columns to un-nest
#columnas_a_desanidar = ['values', 'sub_types']

In [460]:
#y lo que aplico a este otro dataframe es la logica de separacion para json, que estos estaban raros por ese \n
dict_df2 = df_objects[['dependant_types', 'line_items', 'discounts', 'line_item_tiers', 'linked_payments',
'dunning_attempts', 'applied_credits', 'adjustment_credit_notes', 'issued_credit_notes', 'linked_orders', 'billing_address',
'site_details_at_creation', 'tax_origin']]
dict_df2

,dependant_types,line_items,discounts,line_item_tiers,linked_payments,dunning_attempts,applied_credits,adjustment_credit_notes,issued_credit_notes,linked_orders,billing_address,site_details_at_creation,tax_origin
0,{},"[{\n ""id"": ""li_16BdFKULaptqAf1b"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdFKULaptrff1f"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Pablo"",\n ""last_name""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
1,{},"[{\n ""id"": ""li_16BdFKULbAIbu1Ltq"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdFKULbAIdQ1Ltv"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Patricia"",\n ""last_na...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2,{},"[{\n ""id"": ""li_16BdFKULbdrZb261H"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdFKULbdrb5261J"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""David"",\n ""last_name""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
3,{},"[{\n ""id"": ""li_16BdFKULc9w5Q2uIl"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdFKULc9wGj2uJY"",\n ...",[],"[{\n ""applied_amount"": 81,\n ""applied_at...",[],"[{\n ""cn_id"": ""CN-6946"",\n ""cn_reason_co...",[],"{\n ""first_name"": ""Centro Educativo Tres Cu...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
4,{},"[{\n ""id"": ""li_16BdFKULcILx538s4"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""first_name"": ""cliengo-cf"",\n ""valid...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,{},"[{\n ""id"": ""li_16BdHqUMJlDje8CvI"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdHqUMJlDlM8CvN"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Maritza Guadalupe"",\n ...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
576,{},"[{\n ""id"": ""li_16BdHqUMJpRi78KFP"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdHqUMJpRk08KFj"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Felipe"",\n ""last_name...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
577,{},"[{\n ""id"": ""li_AzyjzNUMJuBO2255N"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzyjzNUMJuBPx255S"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""ALBERTO"",\n ""last_nam...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
578,{},"[{\n ""id"": ""li_16BdHqUMKJglH90KC"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BdHqUMKJgmz90KO"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""JOHANNA"",\n ""last_nam...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"


In [78]:
#en la version anterior del archivo, extraia todos los campos tipo json de chargebee, pero en muchas ocasiones eliminaba registros de impuestos o taxes y algunos line items, por lo que en este otro archivo integration2 mejor desanide campo por campo para obtener una data mas precisa y confiable

,dependant_types,line_items,discounts,line_item_tiers,linked_payments,dunning_attempts,applied_credits,adjustment_credit_notes,issued_credit_notes,linked_orders,billing_address,site_details_at_creation,tax_origin


In [461]:
#el codigo desanida el campo de line_items de chargebee
import pandas as pd
import json


df['line_items'] = df['values'].apply(lambda x: json.loads(x).get('line_items') if isinstance(x, str) else x.get('line_items') if isinstance(x, dict) else None)


df_exploded = df.explode('line_items', ignore_index=True)


line_items_df = pd.json_normalize(df_exploded['line_items'])


df_exploded = df_exploded.drop(columns=['line_items'])


df_final = pd.concat([df_exploded, line_items_df], axis=1)


df_final

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,object,subscription_id,customer_id,description,entity_type,entity_id,entity_description,tax_exempt_reason,discount_amount,item_level_discount_amount
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,line_item,Hr550zVRQc5HSu2Epz,5a32d5abe4b008de67485137,Cliengo Starter,plan,CLIENGO_STARTER_CONVER,"Cliengo Starter, 250 conversaciones al mes",tax_not_configured,0,0
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,line_item,Hr550zVRQc5HSu2Epz,5a32d5abe4b008de67485137,Agentes,addon,CLIENGO_STARTER_CONVER_USERS,NaN,tax_not_configured,0,0
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,line_item,AzZdH1SJJkDeO7LJ,5fc70a477b2f58002a63c1ff,Cliengo Starter,plan,CLIENGO_STARTER_50_V2,NaN,tax_not_configured,0,0
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,line_item,AzZdH1SJJkDeO7LJ,5fc70a477b2f58002a63c1ff,Agentes,addon,CLIENGO_STARTER_USERS,NaN,tax_not_configured,0,0
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,line_item,AzZN4TThKsVFII9J,646e44236dc4fd003213cbd1,Cliengo Starter,plan,CLIENGO_STARTER_CONVER,"Cliengo Starter, 250 conversaciones al mes",tax_not_configured,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,line_item,AzZctbRu6XIHz2gRF,5e556f9ee4b097eaa618cd9a,Agentes,addon,CLIENGO_STARTER_USERS,NaN,tax_not_configured,0,0
1229,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,line_item,16A1GIT9dInWv2vrw,6283fedcaf91d9002a23fbf1,Cliengo Starter,plan,CLIENGO_STARTER_50_V2,NaN,tax_not_configured,0,0
1230,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,line_item,16A1GIT9dInWv2vrw,6283fedcaf91d9002a23fbf1,Agentes,addon,CLIENGO_STARTER_USERS,NaN,tax_not_configured,0,0
1231,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,line_item,Hr550zxRQTS3fUm1y,5cacb822e4b0bf154621d24b,Cliengo Starter,plan,CLIENGO_STARTER_CONVER,"Cliengo Starter, 250 conversaciones al mes",tax_not_configured,0,0


In [462]:
#este codigo desanida el campo de discounts de chargebee
df_final['discounts'] = df_final['values'].apply(lambda x: x.get('discounts') if isinstance(x, dict) else None)


def rename_duplicates(df):
    cols = pd.Series(df.columns)
    for dup in df.columns[df.columns.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df


df_final = rename_duplicates(df_final)


df_final_exploded = df_final.explode('discounts', ignore_index=True)


discounts_df = pd.json_normalize(df_final_exploded['discounts']).add_prefix('discounts_')


df_final_exploded = df_final_exploded.drop(columns=['discounts'])
df_final_exploded = df_final_exploded.drop(columns=['line_item_discounts'])


df_discounts = pd.concat([df_final_exploded, discounts_df], axis=1)


df_discounts


,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,tax_exempt_reason,discount_amount,item_level_discount_amount,discounts_object,discounts_entity_type,discounts_description,discounts_amount,discounts_entity_id,discounts_discount_type,discounts_discount_percentage
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1241,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1242,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [463]:
#desanidar el campo de linked payments
df_discounts['linked_payments'] = df_discounts['values'].apply(lambda x: x.get('linked_payments') if isinstance(x, dict) else None)


df_discounts = rename_duplicates(df_discounts)


df_discounts_exploded = df_discounts.explode('linked_payments', ignore_index=True)

payments_df = pd.json_normalize(df_discounts_exploded['linked_payments']).add_prefix('linked_payments_')

df_discounts_exploded = df_discounts_exploded.drop(columns=['linked_payments'])

df_payments = pd.concat([df_discounts_exploded, payments_df], axis=1)

df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,discounts_amount,discounts_entity_id,discounts_discount_type,discounts_discount_percentage,linked_payments_txn_id,linked_payments_applied_amount,linked_payments_applied_at,linked_payments_txn_status,linked_payments_txn_date,linked_payments_txn_amount
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,NaN,NaN,NaN,NaN,txn_16BdFKULaptrff1f,3900.0,1.723790e+09,success,1.723790e+09,3900.0
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,NaN,NaN,NaN,NaN,txn_16BdFKULaptrff1f,3900.0,1.723790e+09,success,1.723790e+09,3900.0
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,NaN,NaN,NaN,NaN,txn_16BdFKULbAIdQ1Ltv,7500.0,1.723795e+09,success,1.723795e+09,7500.0
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,NaN,NaN,NaN,NaN,txn_16BdFKULbAIdQ1Ltv,7500.0,1.723795e+09,success,1.723795e+09,7500.0
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,NaN,NaN,NaN,NaN,txn_16BdFKULbdrb5261J,3900.0,1.723802e+09,success,1.723802e+09,3900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,NaN,NaN,NaN,NaN,txn_AzyjzNUMJuBPx255S,2400.0,1.724456e+09,success,1.724456e+09,2400.0
1779,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,NaN,NaN,NaN,NaN,txn_16BdHqUMKJgmz90KO,7500.0,1.724462e+09,success,1.724462e+09,7500.0
1780,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,NaN,NaN,NaN,NaN,txn_16BdHqUMKJgmz90KO,7500.0,1.724462e+09,success,1.724462e+09,7500.0
1781,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,NaN,NaN,NaN,NaN,txn_16BdHqUMKbmqq9LoU,3900.0,1.724467e+09,success,1.724467e+09,3900.0


In [464]:
#desanida el campo billing address
df_payments['billing_address'] = df_payments['values'].apply(lambda x: x.get('billing_address') if isinstance(x, dict) else None)

billing_address_df = pd.json_normalize(df_payments['billing_address']).add_prefix('billing_address_')

df_payments = df_payments.drop(columns=['billing_address'])

df_payments = pd.concat([df_payments, billing_address_df], axis=1)

df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,billing_address_city,billing_address_state,billing_address_country,billing_address_zip,billing_address_validation_status,billing_address_object,billing_address_email,billing_address_line2,billing_address_phone,billing_address_state_code
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,C.A.B.A.,C.A.B.A.,AR,1414,not_validated,billing_address,NaN,NaN,NaN,NaN
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,C.A.B.A.,C.A.B.A.,AR,1414,not_validated,billing_address,NaN,NaN,NaN,NaN
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,CDMX,NaN,MX,NaN,not_validated,billing_address,NaN,NaN,NaN,NaN
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,CDMX,NaN,MX,NaN,not_validated,billing_address,NaN,NaN,NaN,NaN
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,Rubi,Barcelona,ES,08191,not_validated,billing_address,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,MEXICO,BENITO JUAREZ,MX,03810,not_validated,billing_address,NaN,NaN,NaN,NaN
1779,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,TURRIALBA,LIMÓN,CR,NaN,not_validated,billing_address,NaN,NaN,NaN,NaN
1780,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,TURRIALBA,LIMÓN,CR,NaN,not_validated,billing_address,NaN,NaN,NaN,NaN
1781,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,NaN,NaN,CL,NaN,not_validated,billing_address,NaN,NaN,NaN,NaN


In [465]:
#desanida el campo shipping address
df_payments['shipping_address'] = df_payments['values'].apply(lambda x: x.get('shipping_address') if isinstance(x, dict) else None)

billing_address_df = pd.json_normalize(df_payments['shipping_address']).add_prefix('shipping_address_')

df_payments = df_payments.drop(columns=['shipping_address'])

df_payments = pd.concat([df_payments, billing_address_df], axis=1)

df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,shipping_address_last_name,shipping_address_company,shipping_address_city,shipping_address_country,shipping_address_validation_status,shipping_address_object,shipping_address_state,shipping_address_zip,shipping_address_line1,shipping_address_email
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,GUTIERREZ,SWISS SECURITY BOXES S.A. DE C.V.,MEXICO,MX,not_validated,shipping_address,BENITO JUAREZ,03810,NaN,NaN
1779,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1780,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1781,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [466]:
#desanida el campo line_item_tiers
df_payments['line_item_tiers'] = df_payments['values'].apply(lambda x: x.get('line_item_tiers') if isinstance(x, dict) else None)

df_payments_exploded = df_payments.explode('line_item_tiers', ignore_index=True)

line_item_tiers_df = pd.json_normalize(df_payments_exploded['line_item_tiers']).add_prefix('line_item_tiers_')

df_payments_exploded = df_payments_exploded.drop(columns=['line_item_tiers'])

df_payments = pd.concat([df_payments_exploded, line_item_tiers_df], axis=1)

df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,shipping_address_state,shipping_address_zip,shipping_address_line1,shipping_address_email,line_item_tiers_starting_unit,line_item_tiers_ending_unit,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,NaN,NaN,NaN,NaN,1.0,3.0,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,NaN,NaN,NaN,NaN,1.0,3.0,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2178,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,BENITO JUAREZ,03810,NaN,NaN,1.0,3.0,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P
2179,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,NaN,NaN,NaN,NaN,1.0,3.0,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD
2180,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,NaN,NaN,NaN,NaN,1.0,3.0,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD
2181,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT


In [467]:
#desanida los campos de line_items
df_payments['issued_credit_notes'] = df_payments['values'].apply(lambda x: x.get('issued_credit_notes') if isinstance(x, dict) else None)

df_payments_exploded = df_payments.explode('issued_credit_notes', ignore_index=True)

line_item_tiers_df = pd.json_normalize(df_payments_exploded['issued_credit_notes']).add_prefix('issued_credit_notes_')

df_payments_exploded = df_payments_exploded.drop(columns=['issued_credit_notes'])

df_payments = pd.concat([df_payments_exploded, line_item_tiers_df], axis=1)

df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '186754', 'po_number': 'Argentina', 'cu...",{'line_items': <class 'chargebee.models.invoic...,{},186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '186755', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': '186756', 'po_number': 'España', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,...,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2254,"{'id': '187348', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P,NaN,NaN,NaN,NaN,NaN,NaN
2255,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2256,"{'id': '187349', 'po_number': 'Costa Rica', 'c...",{'line_items': <class 'chargebee.models.invoic...,{},187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2257,"{'id': '187350', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT,NaN,NaN,NaN,NaN,NaN,NaN


In [468]:
#Le cambie la region a argentina utc tal y como lo tuvimos que hacer con panella cuando los traiamos de BigQuery antes
import pytz
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
df_payments['date_from'] = pd.to_datetime(df_payments['date_from'], unit='s', utc=True).dt.tz_convert(argentina_tz).dt.date
df_payments['date_to'] = pd.to_datetime(df_payments['date_to'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['generated_at'] = pd.to_datetime(df_payments['generated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['updated_at'] = pd.to_datetime(df_payments['updated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['paid_at'] = pd.to_datetime(df_payments['paid_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['due_date'] = pd.to_datetime(df_payments['due_date'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['date'] = pd.to_datetime(df_payments['date'], unit='s', utc=True).dt.tz_convert(argentina_tz)

In [469]:
#ordene la data por la invoice_id
order = df_payments.sort_values(by='invoice_id', ascending=False)
order = order.drop(columns=['values', 'sub_types', 'dependant_types'])
order

,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,price_type,date,due_date,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
2258,187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,tax_exclusive,2024-08-23 23:28:41-03:00,2024-08-23 23:28:41-03:00,...,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT,NaN,NaN,NaN,NaN,NaN,NaN
2257,187350,Chile,5cacb822e4b0bf154621d24b,Hr550zxRQTS3fUm1y,True,paid,76280497-2,tax_exclusive,2024-08-23 23:28:41-03:00,2024-08-23 23:28:41-03:00,...,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT,NaN,NaN,NaN,NaN,NaN,NaN
2256,187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,tax_exclusive,2024-08-23 22:16:48-03:00,2024-08-23 22:16:48-03:00,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2255,187349,Costa Rica,6283fedcaf91d9002a23fbf1,16A1GIT9dInWv2vrw,True,paid,110860111,tax_exclusive,2024-08-23 22:16:48-03:00,2024-08-23 22:16:48-03:00,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2254,187348,Mexico,5e556f9ee4b097eaa618cd9a,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,tax_exclusive,2024-08-23 20:35:29-03:00,2024-08-23 20:35:29-03:00,...,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,186756,España,646e44236dc4fd003213cbd1,AzZN4TThKsVFII9J,True,paid,A58075250,tax_exclusive,2024-08-16 06:54:41-03:00,2024-08-16 06:54:41-03:00,...,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I,NaN,NaN,NaN,NaN,NaN,NaN
2,186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,tax_exclusive,2024-08-16 04:57:15-03:00,2024-08-16 04:57:15-03:00,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
3,186755,Mexico,5fc70a477b2f58002a63c1ff,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,tax_exclusive,2024-08-16 04:57:15-03:00,2024-08-16 04:57:15-03:00,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
1,186754,Argentina,5a32d5abe4b008de67485137,Hr550zVRQc5HSu2Epz,True,paid,30680124104,tax_exclusive,2024-08-16 03:36:13-03:00,2024-08-16 03:36:13-03:00,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#siempre renombro los campos de id para que no haya conflicto mas adelante
order = order.rename(columns={'id':'line_item_id'})
order

NameError: name 'order' is not defined

In [3]:
#print_column_types(order)


In [4]:
#a partir de aqui empieza lo de Odoo

In [17]:
#primero con las pruebas de insercion en sales.order


odoo.create('sale.order', [{
                            'campaign_id': False,
                            'source_id': False,
                            'medium_id': False,
                            'activity_ids': [],
                            'activity_state': True,
                            'partner_id': 11, 
                            'company_id': 1,
                            'website_id': 1,
                            }])


356

In [16]:
# aqui yo buscaba antes un id recien creado por el codigo anterior
sale_order_345 = odoo.execute('sale.order', 'search_read', [[['id', '=', 354]], [], 0, 1])
print(sale_order_345)

[{'id': 354, 'campaign_id': False, 'source_id': False, 'medium_id': False, 'activity_ids': [], 'activity_state': False, 'activity_user_id': False, 'activity_type_id': False, 'activity_type_icon': False, 'activity_date_deadline': False, 'my_activity_date_deadline': False, 'activity_summary': False, 'activity_exception_decoration': False, 'activity_exception_icon': False, 'activity_calendar_event_id': False, 'message_is_follower': True, 'message_follower_ids': [1243], 'message_partner_ids': [121], 'message_ids': [2745], 'has_message': True, 'message_needaction': False, 'message_needaction_counter': 0, 'message_has_error': False, 'message_has_error_counter': 0, 'message_attachment_count': 0, 'rating_ids': [], 'website_message_ids': [], 'message_has_sms_error': False, 'access_url': '/my/orders/354', 'access_token': False, 'access_warning': '', 'name': 'V 0001-00000342', 'company_id': [1, 'REGEX LLC'], 'partner_id': [11, 'Gemini Furniture'], 'state': 'draft', 'locked': False, 'client_order_

In [16]:
#aqui hacemos el llamado a las invoices de clientes en Odoo

#odf = odoo.execute('account.move', 'search_read', [[['id', '>', 1]], 0, 20])
#dfc = pd.DataFrame(odf)
#dfc

In [17]:
#este nada mas es una busqueda de una factura en especifico por medio del campo name, 
# que es el que aparece en la interfaz grafica de odoo


#dfc.loc[dfc['name'] == 'BNK1/2024/00007']

In [18]:
#de la funcion que cree al principio se la aplique al dataframe de las invoices de odoo 
# para ver el tipo de datos de cada campo del modelo account.move


#print_column_types(dfc)

In [19]:
#aqui hice el llamado de los campos de tipo float del dataframe para ver los campos que ocupan el equivalente a un valor de moneda
#lo hice para ir viendo los campos que mas adelante debo comparar con los de Chargebee y hacer la migracion correctamente


#floats = dfc.select_dtypes(include=['float64'])
#floats

In [20]:
#este codigo me muestra en un dataframe solamente los campos de tipo float de account.move pero sin excluir el id de la misma
# por eso le hago un concat con el original para que conserve el id

#if 'id' in dfc.columns:
#    floats_with_id = pd.concat([floats, dfc[['id', 'name']]], axis=1)
#else:
#    print("La columna 'id' no existe en el DataFrame original.")

#floats_with_id

In [21]:
#aqui fue en donde vi como es que se almacena el registro de partner_id (que es un array)

#sub_dfc = dfc['partner_id']
#sub_dfc

In [74]:
# Leer todas las facturas (sin establecer un dominio específico) todo para comprobar que la insercion fue exitosa
invoices = odoo.search_read('account.move', [], ['id', 'name', 'date', 'amount_total', 'partner_id'])

customers_invoice = pd.DataFrame(invoices)
customers_invoice


,id,name,date,amount_total,partner_id
0,164,/,2024-09-20,28.75,"[11, Gemini Furniture]"
1,161,INV/2024/00011,2024-09-17,28.75,"[11, Gemini Furniture]"
2,163,/,2024-09-17,0.00,False
3,160,/,2024-09-17,0.00,"[11, Gemini Furniture]"
4,159,/,2024-09-16,28.75,"[11, Gemini Furniture]"
5,158,/,2024-09-16,28.75,"[11, Gemini Furniture]"
6,157,/,2024-09-16,0.00,"[11, Gemini Furniture]"
7,162,INV/2024/00012,2024-09-10,575.00,"[10, Deco Addict]"
8,50,/,2024-08-31,0.00,False
9,156,INV/2024/00010,2024-08-28,28.75,"[11, Gemini Furniture]"


In [75]:
#llame a todas las invoice otra vez pero para separar los campos del array que muestra el partner_id 
# (al final no sirvio porque solamente fue una operacion  de lectura :(  )
i = odoo.search_read('account.move', [], [], limit=None)
inv = pd.DataFrame(i)
inv
#

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,164,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,161,False,False,False,False,False,INV/2024/,11,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,163,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,160,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,159,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
5,158,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
6,157,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
7,162,False,False,False,False,False,INV/2024/,12,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
8,50,False,False,False,False,False,,0,[57],overdue,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
9,156,False,False,False,False,False,INV/2024/,10,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [76]:
#solo mostrar el campo de partner_id para ver como quedan los customer_id de Chargebee  en Odoo
inv[['id', 'partner_id', 'name', 'invoice_date', 'invoice_line_ids']]

,id,partner_id,name,invoice_date,invoice_line_ids
0,164,"[11, Gemini Furniture]",/,False,[]
1,161,"[11, Gemini Furniture]",INV/2024/00011,2024-09-17,[473]
2,163,False,/,False,[479]
3,160,"[11, Gemini Furniture]",/,False,[]
4,159,"[11, Gemini Furniture]",/,False,[]
5,158,"[11, Gemini Furniture]",/,False,[]
6,157,"[11, Gemini Furniture]",/,False,[]
7,162,"[10, Deco Addict]",INV/2024/00012,2024-09-10,[476]
8,50,False,/,False,[]
9,156,"[11, Gemini Furniture]",INV/2024/00010,2024-08-28,[470]


In [471]:
#realice el llamado del campo del customer de Chargebee para ingresarlo en res.partner
customerdf = order['customer_id']
customerdf

2258    5cacb822e4b0bf154621d24b
2257    5cacb822e4b0bf154621d24b
2256    6283fedcaf91d9002a23fbf1
2255    6283fedcaf91d9002a23fbf1
2254    5e556f9ee4b097eaa618cd9a
                  ...           
5       646e44236dc4fd003213cbd1
2       5fc70a477b2f58002a63c1ff
3       5fc70a477b2f58002a63c1ff
1       5a32d5abe4b008de67485137
0       5a32d5abe4b008de67485137
Name: customer_id, Length: 2259, dtype: object

In [472]:
#como los customers que vamos a ingresar estaban duplicados en el llamado, pues elimine los duplicados ya que a partir de este dataframe 
#hare las inserciones en el modelo res.partner
customers_clean = customerdf.drop_duplicates()
customers_clean

2258    5cacb822e4b0bf154621d24b
2256    6283fedcaf91d9002a23fbf1
2254    5e556f9ee4b097eaa618cd9a
2252    6168bafc84869d002a2afe30
2249    663d4a0104d4ea377b73e976
                  ...           
106     6117a8f5748d43002a7f3e55
104     64a812e0a46cbf0032044e4c
4       646e44236dc4fd003213cbd1
2       5fc70a477b2f58002a63c1ff
1       5a32d5abe4b008de67485137
Name: customer_id, Length: 567, dtype: object

In [473]:
#busque los res.partner que es en donde se van a almacenar los customer de Chargebee

i2 = odoo.search_read('res.partner', [], [], limit=None)  # Establecer limit=None para obtener todos los registros
partner = pd.DataFrame(i2)

partner

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,49709,Contact 49709,False,False,False,True,#,True,[157099],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,49683,Contact 49683,False,False,False,True,#,True,[157073],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1324,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1325,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1326,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1327,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [474]:
#deshacerme de los nombres duplicados del modelo res.partner
all_partners=partner.drop_duplicates('name')
all_partners

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,49709,Contact 49709,False,False,False,True,#,True,[157099],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,49683,Contact 49683,False,False,False,True,#,True,[157073],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1324,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1325,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1326,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1327,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [475]:
#visualizo el id y el campo en donde se muestra el nombre del customer que originalmente es el partner id en el modelo res.partner en odoo
i3 = partner[['name', 'id']]
i3 = i3.rename(columns={'name': 'customer_id'})

In [476]:
i3

,customer_id,id
0,(AR) Exento,86
1,(AR) Monotributista,87
2,0h3n-rijaune@yahoo.example.com,67
3,54593f06e4b0d1a073c7326a,49709
4,54dd01dee4b0db69a15ca795,49683
...,...,...
1324,the.jose@gmail.example.com,69
1325,the.real.shane@gmail.example.com,77
1326,thing.thang.thong@gmail.example.com,63
1327,tina.turner@gmail.example.com,76


In [477]:
# Este codigo se encarga de mostrar que los bloques grandes de datos de customer_ids de chargebee que aun no se han insertado en odoo
#esto lo hago para evitar duplicaciones a futuro ya que vamos ingresando toda esa data poco a poco
if isinstance(customers_clean, pd.Series):
    customers_clean = customers_clean.reset_index(drop=True).to_frame(name='customer_id')
else:
    customers_clean = customers_clean.reset_index(drop=True)


i3 = i3.rename(columns={'id': 'partner_id', 'name': 'customer_id'})  # Rename 'id' for clarity in result


merged_df = customers_clean.merge(i3[['customer_id']], on='customer_id', how='left', indicator=True)


unmatched_customers = merged_df[merged_df['_merge'] == 'left_only'][['customer_id']]


unmatched_customers


,customer_id
0,5cacb822e4b0bf154621d24b
1,6283fedcaf91d9002a23fbf1
2,5e556f9ee4b097eaa618cd9a
3,6168bafc84869d002a2afe30
4,663d4a0104d4ea377b73e976
...,...
562,6117a8f5748d43002a7f3e55
563,64a812e0a46cbf0032044e4c
564,646e44236dc4fd003213cbd1
565,5fc70a477b2f58002a63c1ff


In [31]:
#luego ejecute una insercion en ese modelo y funciono exitosamente (el campo que vi que podia llenar con el string de un customer_id)
#era ese campo name de todos los que tiene el dataframe


#odoo.create('res.partner', [{
#    'name': '5dde8435e4b098a27bbd4def'
#}])

In [27]:
#veo los tipos de datos del dataframe de los partners solo con nombre y id
print_column_types(i3)

name 	 object
id 	 int64


In [33]:
#genere un dataframe sobre los campos con tipo de dato object que es un equivalente a un string y poder encontrar que campo habia que
# llenar para que me se pudiera ver el nombre del partner en la interfaz grafica


#many = partner.select_dtypes(include=['object'])
#many

In [478]:
#este codigo se encarga de  utilizar todos los customer_ids de chargebee, enlistandolos para ordenar los campos que se van a insertar line por linea en res.partner
#y asegurandose de que los registros que se vayan a insertar, no existan aun o no esten ya alamacenados en el modelo res.partner de odoo y llevar a cabo la insercion masiva 
# de datos o bulk insert en res.partner

insertion_history = []


def insert_new_partners(unmatched_customers, execution_id):

    existing_partners = odoo.search_read('res.partner', [], ['name'])

    existing_partner_names = {partner['name'] for partner in existing_partners}


    customer_ids = unmatched_customers['customer_id'].tolist()


    records_to_insert = [{'name': customer_id} for customer_id in customer_ids if customer_id not in existing_partner_names]

    for record in records_to_insert:
        odoo.create('res.partner', [record])

    insertion_history.append({
        'execution_id': execution_id,
        'new_records': records_to_insert
    })

    print(f"{len(records_to_insert)} new records have been inserted into res.partner.")


In [479]:

#este otro codigo lo utilizo unicamente con este modelo de res.partner para asegurarme de que cada vez que yo ejecute el bulk insert me genere como un id de ejecucion, es decir que si yo ejecute el bulk inset=rt varias veces, yo pueda llamar a ese bloque de registros insertados segun la vez que yo haya ejecutado ese codigo
#si por ejemplo la primera vez el bulk inserto metio 200 registros, se genera el id de ejecucion conforme el nuemro que vos le pongas y si lo ejecutas por segunda vez
# asegurate de cambiar el numro del parametro execution_id para que python reconozca que lo ejecutaste por segunda vez
def get_inserted_records(execution_id):

    for execution in insertion_history:
        if execution['execution_id'] == execution_id:

            df = pd.DataFrame(execution['new_records'])
            return df


    print(f"No records found for execution ID: {execution_id}")
    return None

In [482]:
#este codigo es el que ejecuta el bulk insert con su respectivo id de ejecucion o la vez que lo ejecutaste
insert_new_partners(unmatched_customers, execution_id='1')

539 new records have been inserted into res.partner.


In [387]:
# Obtener los registros insertados en la ejecución '1'
#esto lo hago para tener el registro de que data acabo de importar para luego tener ese bloque de data para generar las invoices
# o incluso borrarlos cuando lleguemos a otras metodologias
#por ejemplo si lo ejecute 2 veces y quiero ver que registros se insertaron en la segunda ejecucion, me mostrara esos registros
#cabe mencionar de que estos registros se almacenan como datos temporales :(, por lo que si se cierra el archivo y se vuelve a abrir y ejecutar el codigo que aparece a
# continuacion no mostrara nada

df_execution_1 = get_inserted_records('1')
d_exec = pd.DataFrame(df_execution_1)
d_exec

No records found for execution ID: 1


""


In [502]:
#genere este codigo para las ocasiones en las que hay que realizar la eliminacion de muchos registros sin que afecte a los que hay que 
# proteger ya que al hacer el llamado directo y luego borrar es lo que podria sabotear el trabajo debido a un error humano jaja
# Obtener todos los registros del modelo res.partner, ordenados por id de forma descendente y con los campos 'id' y 'name'
i5 = odoo.search_read('res.partner', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame
partner2 = pd.DataFrame(i5)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last = partner2.sort_values(by='id', ascending=False).head(539)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_rows_df = pd.DataFrame(last)
last_rows_df

#antes de los bulk insert habian 118 partners ids y el id del ultimo era 125
#despues del anterior bulk insert quedaban 1329 registros y el ultimo id era 50098

,id,name
142,50637,5a32d5abe4b008de67485137
891,50636,5fc70a477b2f58002a63c1ff
1444,50635,646e44236dc4fd003213cbd1
1464,50634,64a812e0a46cbf0032044e4c
1039,50633,6117a8f5748d43002a7f3e55
...,...,...
1647,50103,663d4a0104d4ea377b73e976
1090,50102,6168bafc84869d002a2afe30
649,50101,5e556f9ee4b097eaa618cd9a
1247,50100,6283fedcaf91d9002a23fbf1


In [486]:
#last_rows_df.drop_duplicates()

In [503]:
#aqui llame al dataframe de los registros de res.partner para ver solamente el nombre y id
p_id = partner[['id', 'name']]
p_id_df = pd.DataFrame(p_id)
p_id_df

,id,name
0,86,(AR) Exento
1,87,(AR) Monotributista
2,67,0h3n-rijaune@yahoo.example.com
3,49709,54593f06e4b0d1a073c7326a
4,49683,54dd01dee4b0db69a15ca795
...,...,...
1324,69,the.jose@gmail.example.com
1325,77,the.real.shane@gmail.example.com
1326,63,thing.thang.thong@gmail.example.com
1327,76,tina.turner@gmail.example.com


In [209]:
#despues uni el dataframe que me mostraba el resultado de los customers que agregue 
# para que hiciera una union con el dataframe de los partners ya que al ser exitosa la insercion de los customers en odoo
# ahora esos customers de chargeBee ya estan en Odoo



#new_partners_df = pd.merge(p_id_df, d_exec, on='name', how='inner')
#new_partners_df

In [230]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados


#partner_id_to_delete = 131  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación


#odoo.execute('res.partner', 'unlink', [partner_id_to_delete])


In [403]:
#Este codigo se encarga de hacer un bulk delete por asi decirlo, elimina el bloque de datos que desees en base al dataframe llamado last_rows_df
partner_ids_to_delete = last['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if partner_ids_to_delete:
    odoo.execute('res.partner', 'unlink', [partner_ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(partner_ids_to_delete)} registros de res.partner.")

{'jsonrpc': '2.0', 'id': 107, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.UserError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in route

Exception: Command has failed

In [271]:
#Buscar un registro de los recien ingresados partners por medio de su id

#last_rows_df.loc[last_rows_df['id'] == 25762]

In [270]:
#Este codigo estaba de prueba para verficar que no se dupliquen los registros y el resultado fue exitoso

#dd = last_rows_df.drop_duplicates('name')
#dd

In [504]:
#renombrar el campo name de res.partner para que se vea como el id del partner o de customer
r_last_rows_df = last_rows_df.rename(columns={'name': 'customer_id'})
r_last_rows_df.sort_values(by='id', ascending=False)
r_last_rows_df

,id,customer_id
142,50637,5a32d5abe4b008de67485137
891,50636,5fc70a477b2f58002a63c1ff
1444,50635,646e44236dc4fd003213cbd1
1464,50634,64a812e0a46cbf0032044e4c
1039,50633,6117a8f5748d43002a7f3e55
...,...,...
1647,50103,663d4a0104d4ea377b73e976
1090,50102,6168bafc84869d002a2afe30
649,50101,5e556f9ee4b097eaa618cd9a
1247,50100,6283fedcaf91d9002a23fbf1


In [505]:
#unir el dataframe anterior con el de order que son todas las invoices de Chargebee y que estos coincidad para luego insertar los
#datos de Chargebee necesarios para migrar a Odoo, terminando en un dataframe que tiene el partner_id y los campos de invoices de
#Chargebee, esto lo hago para que en un solo dataframe tenga toda la data necesaria para posteriormente introducirla en account.move


partners_Odoo_invoices = pd.merge(r_last_rows_df, order, on='customer_id', how='inner')
partners_Odoo_invoices.sort_values(by='id', ascending=False)
partners_Odoo_invoices

,id,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,price_type,date,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,tax_exclusive,2024-08-16 03:36:13-03:00,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
1,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,tax_exclusive,2024-08-16 03:36:13-03:00,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
2,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,tax_exclusive,2024-08-16 04:57:15-03:00,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
3,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,tax_exclusive,2024-08-16 04:57:15-03:00,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
4,50635,646e44236dc4fd003213cbd1,186756,España,AzZN4TThKsVFII9J,True,paid,A58075250,tax_exclusive,2024-08-16 06:54:41-03:00,...,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,50101,5e556f9ee4b097eaa618cd9a,187348,Mexico,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,tax_exclusive,2024-08-23 20:35:29-03:00,...,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P,NaN,NaN,NaN,NaN,NaN,NaN
2029,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,tax_exclusive,2024-08-23 22:16:48-03:00,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2030,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,tax_exclusive,2024-08-23 22:16:48-03:00,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2031,50099,5cacb822e4b0bf154621d24b,187350,Chile,Hr550zxRQTS3fUm1y,True,paid,76280497-2,tax_exclusive,2024-08-23 23:28:41-03:00,...,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT,NaN,NaN,NaN,NaN,NaN,NaN


In [514]:
#order
partners_Odoo_invoices.loc[partners_Odoo_invoices['base_currency_code'] != 'USD']

,id,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,price_type,date,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status


In [49]:
#aqui lo escribo el codigo de insercion para que se vea que funciona al hacerla registro por registro

#aqui el codigo que genera una invoice de clientes 


#odoo.create('account.move', [{
 #   'move_type': 'out_invoice',  # Factura de cliente
  #  'partner_id': 138,  # ID del cliente
   # 'company_id': 1,  # ID de la compañía
    #'website_id': 1,  # ID del sitio web
    #'campaign_id': False,
    #'source_id': False,
    #'medium_id': False,
    #'activity_ids': [],
    #'amount_total': 28.75,
    #'amount_residual': 28.75,
    #'amount_total_signed': 28.75,
    #'amount_total_in_currency_signed': 28.75,
    #'amount_residual_signed': 28.75,
    #'activity_state': True,
#}])

166

In [55]:
#aqui tengo otro codigo de prueba
#
#odoo.create('account.move', [{
 #           'move_type': 'out_invoice',
 #          'partner_id': 34827,
 #           'journal_id': 1,
 #           'currency_id' : 1,
 #           'amount_untaxed': 0.00,
 #           'amount_untaxed_signed': 0.00,
 #           'amount_tax': 0.00,
 #           'amount_tax_signed': 0.00
 #   }])

66064

In [506]:
#aqui yo busque todos los registros de tipo de moneda que tenga registrado account.move para tambien tenerlo en cuenta al insertar las
#invoices de chargebee que esten en pesos o dolares y luego agregarlos a odoo

cur = odoo.search_read('account.move', [], ['currency_id'], limit=None)

# Convertir los resultados a un DataFrame
currency_c = pd.DataFrame(cur)
currency_c

,id,currency_id
0,164,"[1, USD]"
1,161,"[1, USD]"
2,163,"[1, USD]"
3,160,"[1, USD]"
4,159,"[1, USD]"
...,...,...
1352,19,"[1, USD]"
1353,18,"[1, USD]"
1354,17,"[1, USD]"
1355,11,"[1, USD]"


In [426]:
# Revisaso la estructura de los datos del campo para poderlo separar en 2, luego unirlo con el campo de currency de chargebee y poder 
# unir ambos dataframes por medio del tipo de moneda mas adelante

print(currency_c['currency_id'].head(10))  # Revisar si los valores están en el formato correcto

0    [1, USD]
1    [1, USD]
2    [1, USD]
3    [1, USD]
4    [1, USD]
5    [1, USD]
6    [1, USD]
7    [1, USD]
8    [1, USD]
9    [1, USD]
Name: currency_id, dtype: object


In [5]:
#aqui busco entre todas las invoices de chargebee, del dataframe en el que ordene todos por el id de la invoice de chargebee
#  para ver que invoices de chargebee tienen otra moneda diferente a dolares 
#order.loc[order['currency_code'] != 'USD']

In [507]:
#Este codigo lo use para separar el campo currency_id de account.move en 2 ya que aparece su id como un array que tiene un int y un 
# string como se mostro en los codigos anteriores que llaman a ese campo

# Asegurarse de que currency_id sean listas y cada lista tenga dos elementos
currency_c['currency_id'] = currency_c['currency_id'].apply(lambda x: x if isinstance(x, list) and len(x) == 2 else [None, None])

# Separar la columna currency_id en dos columnas: currency_num y currency_name
currency_c[['currency_num', 'currency_code']] = pd.DataFrame(currency_c['currency_id'].tolist(), index=currency_c.index)

# Convertir el campo currency_num a tipo numérico
currency_c['currency_num'] = pd.to_numeric(currency_c['currency_num'], errors='coerce')

# Verificar el resultado refinado con las nuevas columnas
currency_refined = currency_c[['currency_num', 'currency_code']]
currency_refined

,currency_num,currency_code
0,1,USD
1,1,USD
2,1,USD
3,1,USD
4,1,USD
...,...,...
1352,1,USD
1353,1,USD
1354,1,USD
1355,1,USD


In [508]:
#una vez se determinan los campos de moneda, se eliminan los duplicados para ver que tipo de moneda tiene registrado odoo en las facturas
currency_re_defined = currency_refined.drop_duplicates('currency_code')
currency_re_defined

,currency_num,currency_code
0,1,USD


In [516]:
#luego uni el dataframe que mostraba el tipo de moneda registrado en odoo con las invoices de chargebee para si asegurar de que la data que tenga pagos en USD o dolares se 
# inserten en el modelo account.move de odoo
partners_to_odoo_invoices = pd.merge(currency_re_defined, partners_Odoo_invoices, on='currency_code', how='right')
partners_to_odoo_invoices

,currency_num,currency_code,id,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,1.0,USD,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,USD,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,USD,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,USD,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,USD,50635,646e44236dc4fd003213cbd1,186756,España,AzZN4TThKsVFII9J,True,paid,A58075250,...,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,1.0,USD,50101,5e556f9ee4b097eaa618cd9a,187348,Mexico,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P,NaN,NaN,NaN,NaN,NaN,NaN
2029,1.0,USD,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2030,1.0,USD,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2031,1.0,USD,50099,5cacb822e4b0bf154621d24b,187350,Chile,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT,NaN,NaN,NaN,NaN,NaN,NaN


In [517]:
#aqui renombre el campo de id que tenia el modelo de res.partner de odoo que ya estaba unido con las invoices de chargebee para que en base a ese unico dataframe se inserte 
# la data en account.move
renamed_partners_to_odoo = partners_to_odoo_invoices.rename(columns={'id': 'PartnerID'})
renamed_partners_to_odoo

,currency_num,currency_code,PartnerID,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,1.0,USD,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,USD,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,USD,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,USD,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,USD,50635,646e44236dc4fd003213cbd1,186756,España,AzZN4TThKsVFII9J,True,paid,A58075250,...,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,1.0,USD,50101,5e556f9ee4b097eaa618cd9a,187348,Mexico,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P,NaN,NaN,NaN,NaN,NaN,NaN
2029,1.0,USD,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2030,1.0,USD,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN
2031,1.0,USD,50099,5cacb822e4b0bf154621d24b,187350,Chile,Hr550zxRQTS3fUm1y,True,paid,76280497-2,...,1.0,0.0,line_item_tier,li_16BdHqUMKbmpN9LoT,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
#renamed_partners_to_odoo.loc[renamed_partners_to_odoo['partner_id'] == 0]

In [224]:
#visualizar si algun registro de facturas de agosto tiene impuestos

#partners_to_odoo_invoices.loc[partners_to_odoo_invoices['tax_amount'] > 0]

In [404]:
#aqui llame a un dataframe que declare mas abajo, ignora esto

# date_df

In [110]:
#Este dataframe lo use unicamente para Filtrar las columnas que contienen 'date' en su nombre
date_columns = [col for col in order.columns if 'date' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
date_df = order[date_columns]

# Mostrar el nuevo dataframe filtrado
date_df


,date,due_date,updated_at,expected_payment_date,date_from,date_to,linked_payments_txn_date,issued_credit_notes_cn_date
2896,2024-08-08 02:21:06-03:00,2024-08-08 02:21:06-03:00,2024-08-13 02:21:11-03:00,NaN,2024-08-08,2024-09-08 02:21:06-03:00,1.723526e+09,NaN
2895,2024-08-08 02:21:06-03:00,2024-08-08 02:21:06-03:00,2024-08-13 02:21:11-03:00,NaN,2024-08-08,2024-09-08 02:21:06-03:00,1.723267e+09,NaN
2894,2024-08-08 02:21:06-03:00,2024-08-08 02:21:06-03:00,2024-08-13 02:21:11-03:00,NaN,2024-08-08,2024-09-08 02:21:06-03:00,1.723094e+09,NaN
2893,2024-08-08 02:21:06-03:00,2024-08-08 02:21:06-03:00,2024-08-13 02:21:11-03:00,NaN,2024-08-08,2024-09-08 02:21:06-03:00,1.723526e+09,NaN
2892,2024-08-08 02:21:06-03:00,2024-08-08 02:21:06-03:00,2024-08-13 02:21:11-03:00,NaN,2024-08-08,2024-09-08 02:21:06-03:00,1.723267e+09,NaN
...,...,...,...,...,...,...,...,...
2,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
5,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
3,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
1,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,2024-08-23 14:55:19-03:00,NaN,2024-08-01,2024-09-01 05:08:00-03:00,1.724349e+09,NaN


In [314]:
# Filtrar las columnas que contienen 'tax' en su nombre
tax_columns = [col for col in inv.columns if 'tax' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'tax'
tax = inv[tax_columns]

# Mostrar el nuevo dataframe filtrado
tax


,is_tax_computed_externally,is_avatax,tax_cash_basis_rec_id,tax_cash_basis_origin_move_id,tax_cash_basis_created_move_ids,always_tax_exigible,tax_calculation_rounding_method,amount_untaxed,amount_tax,amount_untaxed_signed,...,tax_totals,tax_lock_date_message,tax_country_id,tax_country_code,avatax_unique_code,avatax_tax_date,tax_closing_end_date,tax_report_control_error,tax_closing_alert,tax_closing_show_multi_closing_warning
0,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 164,False,False,False,False,False
1,False,False,False,False,[],False,round_per_line,25.0,3.75,25.0,...,"{'amount_untaxed': 25.0, 'amount_total': 28.75...",False,"[233, United States]",US,Journal Entry 161,False,False,False,False,False
2,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 163,False,False,False,False,False
3,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 160,False,False,False,False,False
4,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 159,False,False,False,False,False
5,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 158,False,False,False,False,False
6,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 157,False,False,False,False,False
7,False,False,False,False,[],False,round_per_line,500.0,75.00,500.0,...,"{'amount_untaxed': 500.0, 'amount_total': 575....",False,"[233, United States]",US,Journal Entry 162,False,False,False,False,False
8,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 50,False,2024-08-31,False,False,False
9,False,False,False,False,[],False,round_per_line,25.0,3.75,25.0,...,"{'amount_untaxed': 25.0, 'amount_total': 28.75...",False,"[233, United States]",US,Journal Entry 156,False,False,False,False,False


In [6]:
renamed_partners_to_odoo

NameError: name 'renamed_partners_to_odoo' is not defined

In [519]:
#este otro codigo se encarga de eliminar los duplicados que se hayan  realizado debido a las uniones entre dataframes, la duplicacion a eliminar la agregue en base al invoice id ya que la cantidad de invoices que tiene chargebee es la misma cantidad que vamos a insertar en odo(siempre iendo bloque por bloque)
clean_renamed_partners_to_odoo = renamed_partners_to_odoo.drop_duplicates('invoice_id')
clean_renamed_partners_to_odoo

,currency_num,currency_code,PartnerID,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,1.0,USD,50637,5a32d5abe4b008de67485137,186754,Argentina,Hr550zVRQc5HSu2Epz,True,paid,30680124104,...,1.0,0.0,line_item_tier,li_16BdFKULaptqGf1c,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,USD,50636,5fc70a477b2f58002a63c1ff,186755,Mexico,AzZdH1SJJkDeO7LJ,True,paid,DPL060130RE6,...,3.0,0.0,line_item_tier,li_16BdFKULbAIbz1Ltr,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,USD,50635,646e44236dc4fd003213cbd1,186756,España,AzZN4TThKsVFII9J,True,paid,A58075250,...,1.0,0.0,line_item_tier,li_16BdFKULbdrZf261I,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,USD,50634,64a812e0a46cbf0032044e4c,186758,CUENTA TEST,16CaLfTvmAKjx1inK,True,paid,None,...,1.0,0.0,line_item_tier,li_16BdFKULcILxB38s6,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,USD,50633,6117a8f5748d43002a7f3e55,186759,Argentina,AzqSEjSxbgMC998e,True,paid,30671644197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,1.0,USD,50103,663d4a0104d4ea377b73e976,187346,Mexico,16BiiCUDdoBbY6l7c,True,paid,ROCM700304L95,...,1.0,0.0,line_item_tier,li_16BdHqUMJlDji8CvJ,NaN,NaN,NaN,NaN,NaN,NaN
2025,1.0,USD,50102,6168bafc84869d002a2afe30,187347,Mexico,16A16fSyJAFdS2oAf,True,paid,BAMF660710BK3,...,6.0,0.0,line_item_tier,li_16BdHqUMJpRiC8KFR,NaN,NaN,NaN,NaN,NaN,NaN
2027,1.0,USD,50101,5e556f9ee4b097eaa618cd9a,187348,Mexico,AzZctbRu6XIHz2gRF,True,paid,SSB120314V52,...,3.0,0.0,line_item_tier,li_AzyjzNUMJuBO8255P,NaN,NaN,NaN,NaN,NaN,NaN
2029,1.0,USD,50100,6283fedcaf91d9002a23fbf1,187349,Costa Rica,16A1GIT9dInWv2vrw,True,paid,110860111,...,3.0,0.0,line_item_tier,li_16BdHqUMKJglM90KD,NaN,NaN,NaN,NaN,NaN,NaN


In [522]:
#una vez teniendo el modelo de res.partner e identificado los datos que debo insertar, generare la insercion de invoices en Odoo
#solamente como prueba ya que unicamente voy a incluir unos campos a las facturas pero lo mas importante en este caso es ingresar el
# partner_id de cada partner que ya enliste despues de ingresar el monton de registros de Chargebee en Odoo y asi de una vez saber a que
#partners debo ingresar en las invoices y tambien la fecha y el tipo de factura que en este caso son todas de clientes
# Función para insertar facturas en account.move con validación de datos
import signal
import time

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_invoices_with_rollback(clean_renamed_partners_to_odoo, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []

    # Iterar sobre el DataFrame
    for _, row in clean_renamed_partners_to_odoo.iterrows():
        record = {
            'move_type': 'out_invoice',
            'partner_id': row['PartnerID'],
            'journal_id': 1,
            'currency_id': row['currency_num'],
            'invoice_date': row['date'].strftime('%Y-%m-%d'),
            'amount_tax': row['tax_amount']
        }

        if 'partner_id' not in record or not record['partner_id']:
            print(f"Registro inválido: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro (necesitamos envolver el diccionario en una lista)
                result = odoo.execute('account.move', 'create', [record])
                inserted_record_ids.append(result)

            insertion_history.append({
                'execution_id': execution_id,
                'new_records': inserted_record_ids
            })

            print(f"Se insertaron {len(inserted_record_ids)} nuevas facturas en account.move.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.")


In [525]:
# Prueba de la función de inserción
insert_invoices_with_rollback(clean_renamed_partners_to_odoo, execution_id='4')

Error durante la inserción: Out of range float values are not JSON compliant: nan
Intentando eliminar los registros insertados debido al error...
{'jsonrpc': '2.0', 'id': 476, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'builtins.TypeError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/

In [527]:
#codigo para mostrar los ultimos registros en el modelo, tomando en cuenta la cantidad d eregistros que se insertaron en el Bulk insert anterior

#amount_untaxed amount_tax
i6 = odoo.search_read('account.move', [], ['id', 'partner_id', 'currency_id', 'amount_untaxed', 'amount_tax'], limit=None)

# Convertir los resultados a un DataFrame
inv_df = pd.DataFrame(i6)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_inv = inv_df.sort_values(by='id', ascending=False).head(474)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices = pd.DataFrame(last_inv)
last_invoices
#habian 39 registros antes de la insercion y el id del ultimo registro antes de la insercion es el 164
#el final de la insercion anterior habian 738 registros y el ultimo registro tenia el id 106431

,id,partner_id,currency_id,amount_untaxed,amount_tax
12,107523,"[50168, 5e8609a0e4b07bd91f6abfd2]","[1, USD]",0.00,0.0
13,107522,"[50169, 641dae8cbcaed5002571ecd4]","[1, USD]",0.00,0.0
14,107521,"[50170, 636c1f04f68c5d002afe3036]","[1, USD]",0.00,0.0
15,107520,"[50171, 5e7104afe4b0f529d0181474]","[1, USD]",0.00,0.0
16,107519,"[50172, 5cc33d7be4b0f4d3ee9627cd]","[1, USD]",0.00,0.0
...,...,...,...,...,...
481,107054,"[50634, 64a812e0a46cbf0032044e4c]","[1, USD]",0.00,0.0
482,107053,"[50635, 646e44236dc4fd003213cbd1]","[1, USD]",0.00,0.0
483,107052,"[50636, 5fc70a477b2f58002a63c1ff]","[1, USD]",0.00,0.0
484,107051,"[50637, 5a32d5abe4b008de67485137]","[1, USD]",0.00,0.0


In [444]:

#este codigo se encarga de separar el array que esta en el campo de partner_id ya que lo usaremos mas adelante
last_invoices[['partnerID', 'customer_id']] = pd.DataFrame(last_invoices['partner_id'].tolist(), index=last_invoices.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices['partnerID'] = pd.to_numeric(last_invoices['partnerID'], errors='coerce')

# Verificar el resultado
last_invoices = last_invoices[['id', 'partnerID', 'customer_id']]
last_invoices

,id,partnerID,customer_id
551,107050,48810,5be3127ae4b0e9b0e27b3220
552,107049,48810,5be3127ae4b0e9b0e27b3220
553,107048,48810,5be3127ae4b0e9b0e27b3220
554,107047,48810,5be3127ae4b0e9b0e27b3220
555,107046,48810,5be3127ae4b0e9b0e27b3220
...,...,...,...
636,106436,50094,5e9f92ede4b0d2f7dde0bd82
637,106435,50095,5d9cca09e4b000472fe908c5
638,106434,50096,5b086bcde4b0cd31b95a484d
639,106433,50097,656636ecdc1caa00320f5e89


In [324]:
#last_invoices.loc[last_invoices['amount_tax'] > 0]

In [290]:
#pppppp = order[['customer_id', 'total']]
#pppppp

In [291]:
#pppppp.loc[pppppp['customer_id'] == '60c3d852b0d246002afdde1b']

In [292]:
#confirmar que no tengan impuestos ya que las facturas de agosto no tienen impuestos

#last_invoices.loc[last_invoices['amount_untaxed'] > 0]

In [336]:
# Obtener los IDs de los últimos  registros de account.move y eliminarlos en base al lsitado de los ultimos ids o de un bloque de
#registros a eliminar que se basa en el dataframe de last_invoices que ejecute anteriormente y de la misma manera que en res.partner
ids_to_delete = last_inv['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if ids_to_delete:
    odoo.execute('account.move', 'unlink', [ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(ids_to_delete)} registros de account.move.")

Se han eliminado 107 registros de account.move.


In [445]:
#unir los dataframes de los partners que se insertaron  para ver a que customer pertnece cada partner y que invoice_id de Odoo tiene ese partner para evitar faltas de 
# registros
#last_invoices
lines_join = pd.merge(renamed_partners_to_odoo, last_invoices, on='customer_id', how='inner')
lines_join

,currency_num,currency_code,PartnerID,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status,id,partnerID
0,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
1,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
2,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
3,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
4,1,USD,50096,5b086bcde4b0cd31b95a484d,186115,Argentina - Transferencia Bancaria,AzqBuBU5Q0tWME66L,True,paid,None,...,line_item_tier,li_16BdQ2UKr766V24O2,NaN,NaN,NaN,NaN,NaN,NaN,106434,50096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,1,USD,48810,5be3127ae4b0e9b0e27b3220,186140,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16BamCUKsLNOs3B1Z,CN-6937,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107030,48810
3122,1,USD,48810,5be3127ae4b0e9b0e27b3220,186140,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16BamCUKsLNOs3B1Z,CN-6937,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107029,48810
3123,1,USD,48810,5be3127ae4b0e9b0e27b3220,186140,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16BamCUKsLNOs3B1Z,CN-6937,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107028,48810
3124,1,USD,48810,5be3127ae4b0e9b0e27b3220,186140,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16BamCUKsLNOs3B1Z,CN-6937,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107027,48810


In [447]:
#asegurarme de eliminar las duplicaciones de line_items con sus respectivos invoices para no insertar line_items duplicados
lines_join = lines_join.drop_duplicates(['line_item_id', 'invoice_id'])
lines_join

,currency_num,currency_code,PartnerID,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status,id,partnerID
0,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
1,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
2,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
3,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
4,1,USD,50096,5b086bcde4b0cd31b95a484d,186115,Argentina - Transferencia Bancaria,AzqBuBU5Q0tWME66L,True,paid,None,...,line_item_tier,li_16BdQ2UKr766V24O2,NaN,NaN,NaN,NaN,NaN,NaN,106434,50096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,1,USD,48810,5be3127ae4b0e9b0e27b3220,186144,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzywoMUKsLPWG3GqF,CN-6911,subscription_change,Subscription Change,1.723164e+09,241.0,refunded,107050,48810
3026,1,USD,48810,5be3127ae4b0e9b0e27b3220,186143,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16CGXtUKsLOkV38nH,CN-6912,subscription_change,Subscription Change,1.723164e+09,241.0,refunded,107050,48810
3051,1,USD,48810,5be3127ae4b0e9b0e27b3220,186142,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzqYEmUKsLNyp3BsY,CN-6916,subscription_change,Subscription Change,1.723230e+09,217.0,refunded,107050,48810
3076,1,USD,48810,5be3127ae4b0e9b0e27b3220,186141,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzZuCUUKsLNjY3CfM,CN-6936,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107050,48810


In [448]:
#aqui nada mas ordeno todas las line_items de cada factura en base al orden de los clientes ya insertados en res.partner y que ya tienen sus facturas en account.move y
# finalmente hacer un bulk insert en account.move.line
ordered_lines_join = lines_join.sort_values(by='PartnerID', ascending=False)
ordered_lines_join

,currency_num,currency_code,PartnerID,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status,id,partnerID
0,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
1,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
2,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
3,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
5,1,USD,50096,5b086bcde4b0cd31b95a484d,186115,Argentina - Transferencia Bancaria,AzqBuBU5Q0tWME66L,True,paid,None,...,line_item_tier,li_16BdQ2UKr766V24O2,NaN,NaN,NaN,NaN,NaN,NaN,106434,50096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,1,USD,48810,5be3127ae4b0e9b0e27b3220,186144,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzywoMUKsLPWG3GqF,CN-6911,subscription_change,Subscription Change,1.723164e+09,241.0,refunded,107050,48810
3026,1,USD,48810,5be3127ae4b0e9b0e27b3220,186143,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16CGXtUKsLOkV38nH,CN-6912,subscription_change,Subscription Change,1.723164e+09,241.0,refunded,107050,48810
3051,1,USD,48810,5be3127ae4b0e9b0e27b3220,186142,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzqYEmUKsLNyp3BsY,CN-6916,subscription_change,Subscription Change,1.723230e+09,217.0,refunded,107050,48810
3076,1,USD,48810,5be3127ae4b0e9b0e27b3220,186141,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzZuCUUKsLNjY3CfM,CN-6936,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107050,48810


In [300]:
#corrigiendo datos gracias a un customer con duplicados

#ddaddadaf = lines_join_clean.loc[lines_join_clean['customer_id'] == '6538aee581e02900329122e6']
#ddaddadaf

In [250]:
#ddaddadaf[['customer_id', 'total']]

In [449]:
#aqui me encargo de renombrar campos de el registro de las ultimas invoices insertadas para poderlo unir con otro dataframe mas adelante
last_moves = last_invoices.rename(columns={'id': 'move_id_num'})
last_moves

,move_id_num,partnerID,customer_id
551,107050,48810,5be3127ae4b0e9b0e27b3220
552,107049,48810,5be3127ae4b0e9b0e27b3220
553,107048,48810,5be3127ae4b0e9b0e27b3220
554,107047,48810,5be3127ae4b0e9b0e27b3220
555,107046,48810,5be3127ae4b0e9b0e27b3220
...,...,...,...
636,106436,50094,5e9f92ede4b0d2f7dde0bd82
637,106435,50095,5d9cca09e4b000472fe908c5
638,106434,50096,5b086bcde4b0cd31b95a484d
639,106433,50097,656636ecdc1caa00320f5e89


In [302]:
#una busqueda de registro por id


#last_moves.loc[last_moves['move_id_num'] == 20097]

In [303]:
print_column_types(last_invoices)

id 	 int64
partner_id 	 object
currency_id 	 object
amount_untaxed 	 float64
amount_tax 	 float64
PartnerID 	 int64
name 	 object


In [333]:
#order

In [100]:
#a partir del dataframe de Chargebee, me toca aislar todos los campos que incluyen los line items, descuentos y notas de credito

Chargebee_li = order[['id_line_items_expanded', 'date_from', 'date_to', 'unit_amount', 'quantity', 'amount', 'pricing_model', 'is_taxed',
       'tax_amount', 'object_line_items_expanded', 'description', 'entity_type', 'entity_id', 'entity_description', 'tax_exempt_reason'
       , 'discount_amount', 'item_level_discount_amount', 'starting_unit', 'ending_unit', 'quantity_used',
       'unit_amount_line_item_tiers_expanded', 'line_item_id', 'txn_id', 'applied_amount', 'applied_at', 'txn_status', 'txn_date',
       'txn_amount', 'applied_amount_applied_credits_expanded', 'applied_at_applied_credits_expanded', 'cn_id', 'cn_reason_code',
       'cn_create_reason_code', 'cn_date', 'cn_status', 'cn_id_adjustment_credit_notes_expanded',
       'cn_reason_code_adjustment_credit_notes_expanded', 'cn_create_reason_code_adjustment_credit_notes_expanded',
       'cn_date_adjustment_credit_notes_expanded', 'cn_total', 'cn_status_adjustment_credit_notes_expanded',
       'cn_id_issued_credit_notes_expanded', 'cn_reason_code_issued_credit_notes_expanded',
       'cn_create_reason_code_issued_credit_notes_expanded', 'cn_date_issued_credit_notes_expanded',
       'cn_total_issued_credit_notes_expanded', 'cn_status_issued_credit_notes_expanded', 'created_at', 'attempt', 'dunning_type',
       'transaction_id', 'txn_status_dunning_attempts_expanded', 'txn_amount_dunning_attempts_expanded', 'object_discounts_expanded',
       'entity_type_discounts_expanded', 'description_discounts_expanded', 'amount_discounts_expanded', 'discount_type',
       'discount_percentage', 'object_line_item_discounts_expanded', 'line_item_id_line_item_discounts_expanded',
       'discount_type_line_item_discounts_expanded', 'discount_amount_line_item_discounts_expanded', 'coupon_id',
       'entity_id_line_item_discounts_expanded', 'note', 'entity_type_notes_expanded', 'entity_type_notes_expanded']]

In [101]:
cbli = Chargebee_li.drop_duplicates('id_line_items_expanded')
cbli

,id_line_items_expanded,date_from,date_to,unit_amount,quantity,amount,pricing_model,is_taxed,tax_amount,object_line_items_expanded,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
0,li_6olL2UKBV7cg22wd,2024-08-01,2024-09-01 05:08:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,li_6olL2UKBV7cm22we,2024-08-01,2024-09-01 05:08:00-03:00,0,1,0,tiered,False,0,line_item,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,li_Azz6aoUKC1bN11M2p,2024-08-01,2024-09-01 07:17:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,li_Azz6aoUKC1bN81M2q,2024-08-01,2024-09-01 07:17:00-03:00,750,4,3000,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,li_AzyjxKUKC78gO1LOQ,2024-08-01,2024-09-01 07:39:00-03:00,39,100,3900,stairstep,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,li_AzZnNZULMe4v63Smj,2024-08-13,2024-09-13 17:21:28-03:00,3000,1,3000,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2451,li_16BdPRULMXR8j76Yg,2024-08-13,2024-09-13 16:55:04-03:00,48,50,2400,stairstep,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2452,li_16BdPRULMXR8o76Yh,2024-08-13,2024-09-13 16:55:04-03:00,0,3,0,tiered,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2444,li_16BdPRULMPptF6sd4,2024-08-13,2024-09-13 16:24:52-03:00,467,3,1400,tiered,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
# Obtener la descripción del modelo 'account.move.line'
fields_description = odoo.execute('account.move.line', 'fields_get', [], {'attributes': ['String', 'type']})
lines = pd.DataFrame(fields_description)
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
# Mostrar las columnas (campos) del modelo
for field, description in fields_description.items():
    print(f"Campo: {field}, Tipo: {description['type']}, Descripción: {description['string']}")

KeyError: 'string'

In [180]:
# Obtener todos los campos del modelo account.move.line que lleven taxes
fields_move_line = odoo.execute('account.move.line', 'fields_get', {})

# Filtrar los campos que contengan 'tax' o 'tx' y extraer solo los nombres de los campos
filtered_fields = [field for field in fields_move_line.keys() if 'tax' in field or 'tx' in field]

# Mostrar los campos filtrados
taxes_df = pd.DataFrame(filtered_fields)
taxes_df

,0
0,tax_ids
1,group_tax_id
2,tax_line_id
3,tax_group_id
4,tax_base_amount
5,tax_repartition_line_id
6,tax_tag_ids
7,tax_tag_invert
8,tax_calculation_rounding_method
9,tax_key


In [70]:
#tax_ids
#group_tax_id
#tax_line_id
#tax_group_id
#tax_base_amount
#tax_repartition_line_id
#tax_tag_ids
#tax_tag_invert
#tax_calculation_rounding_method
#tax_key
#compute_all_tax
#compute_all_tax_dirty
#non_deductible_tax_value
#tax_settlement_move_id
#tax_state

In [450]:
#ejemplo de creacion de line_item (uno a la vez)

#odoo.create('account.move.line', [{
#    'move_id': 66064,  # ID del movimiento contable (relacionado con account.move)
#    #'account_id': 3,  # ID de la cuenta contable
#    'name': 'Venta de producto',  # Descripción de la línea
#    'quantity': 1.0,  # Cantidad
#    'price_unit': 28.75,  # Precio unitario
#    'debit': 0.0,  # Débito
#    #'credit': 28.75,  # Crédito
#    'partner_id': 34827,  # ID del cliente
#    #'company_id': 1,  # ID de la compañía
#    'currency_id': 1,  # ID de la moneda
#    #'date_maturity': '2024-09-30',  # Fecha de vencimiento
#}])


#y llamo nuevamente al dataframe que usare de referencia para hacer el bulk insert en account.move.line
ordered_lines_join

,currency_num,currency_code,PartnerID,customer_id,invoice_id,po_number,subscription_id,recurring,status,vat_number,...,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status,id,partnerID
0,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
1,1,USD,50098,6183f38cc4176e002a26ce0d,186113,España,169noMTdeMPuqKRb,True,paid,ESB65641334,...,line_item_tier,li_16BdQ2UKqNg8VtKT,NaN,NaN,NaN,NaN,NaN,NaN,106432,50098
2,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
3,1,USD,50097,656636ecdc1caa00320f5e89,186114,España,AzZN2NU0lchfM1iPj,True,paid,B58370990,...,line_item_tier,li_16BdQ2UKr5Nni21S2,NaN,NaN,NaN,NaN,NaN,NaN,106433,50097
5,1,USD,50096,5b086bcde4b0cd31b95a484d,186115,Argentina - Transferencia Bancaria,AzqBuBU5Q0tWME66L,True,paid,None,...,line_item_tier,li_16BdQ2UKr766V24O2,NaN,NaN,NaN,NaN,NaN,NaN,106434,50096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,1,USD,48810,5be3127ae4b0e9b0e27b3220,186144,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzywoMUKsLPWG3GqF,CN-6911,subscription_change,Subscription Change,1.723164e+09,241.0,refunded,107050,48810
3026,1,USD,48810,5be3127ae4b0e9b0e27b3220,186143,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_16CGXtUKsLOkV38nH,CN-6912,subscription_change,Subscription Change,1.723164e+09,241.0,refunded,107050,48810
3051,1,USD,48810,5be3127ae4b0e9b0e27b3220,186142,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzqYEmUKsLNyp3BsY,CN-6916,subscription_change,Subscription Change,1.723230e+09,217.0,refunded,107050,48810
3076,1,USD,48810,5be3127ae4b0e9b0e27b3220,186141,Mexico,1mkVvw2RCFlS8NxFf,True,paid,CET0810145K6,...,line_item_tier,li_AzZuCUUKsLNjY3CfM,CN-6936,subscription_change,Subscription Change,1.723744e+09,25.0,refunded,107050,48810


In [452]:
#este codigo se encarga de hacer el bulk insert en account.move.line, utilizando la misma logica que los anteriores bulk inserts en los otros modelos
import signal
import time


interrupted = False


def handle_interrupt(signum, frame):
    global interrupted
    print(f"Signal {signum} received. Initiating rollback...")
    interrupted = True  # Mark interruption
    raise Exception("Process interrupted manually or by the system.")


signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_account_move_lines_with_rollback(ordered_lines_join, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []

    existing_move_ids = odoo.search_read('account.move.line', [], ['move_id'])
    existing_move_ids_set = set([item['move_id'][0] for item in existing_move_ids if isinstance(item['move_id'], list)])

    for _, row in ordered_lines_join.iterrows():
        move_id = row['id']
        partner_id = row['PartnerID']
        name = row['description']
        quantity = row['quantity']
        amount = row['amount']
        tax = row['tax']

        record = {
            'move_id': move_id,
            'name': name,           # Description of the line
            'quantity': quantity,   # Quantity
            'price_unit': amount / 100,  # Unit price
            'partner_id': partner_id,    # Partner ID
            'currency_id': 1,       # Currency ID
            'tax_base_amount': tax / 100
        }


        if 'move_id' not in record or not record['move_id']:
            print(f"Invalid record: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interruption detected before insertion.")

                result = odoo.execute('account.move.line', 'create', [record])
                inserted_record_ids.append(result)

            print(f"{len(inserted_record_ids)} new lines were inserted into account.move.line.")

        except Exception as e:
            print(f"Error during insertion: {str(e)}")

            if inserted_record_ids:
                try:
                    print("Attempting to delete inserted records due to error...")
                    odoo.execute('account.move.line', 'unlink', inserted_record_ids)
                    print("Records successfully deleted.")
                except Exception as rollback_error:
                    print(f"Error when attempting to delete records: {str(rollback_error)}")

    else:
        print("No valid records found for insertion into account.move.line.")


In [454]:
# Ejecutar el bulk insert
insert_account_move_lines_with_rollback(ordered_lines_join, execution_id='1')

1302 new lines were inserted into account.move.line.


In [455]:
#este otro codigo al igual que con los modelos anteriores, se encarga de leer los regitros recien insertados en el modelo que en este caso es account.move.line
#que en este caso se duplicaban, es decir yo insertaba por ejemplo 1000 registros y al final esos mil se duplicaban y tenia que ver los ultimos 2000
#pero esto me lo explico augusto de que al tener por default un campo de debito o credito se generaba nuevamente el registro como registro con debito
i7 = odoo.search_read('account.move.line', [], ['id', 'move_id', 'tax_base_amount', 
'compute_all_tax_dirty', 'non_deductible_tax_value', 'tax_settlement_move_id', 'tax_state', 'partner_id', 'price_unit'], limit=None)

# Convertir los resultados a un DataFrame
df_line_items = pd.DataFrame(i7)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_li_inv = df_line_items.sort_values(by='id', ascending=False).head(5000)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices_lines = pd.DataFrame(last_li_inv)
last_invoices_lines

#antesde iniciar las inserciones masivas habian 94 registros
#despues de una insercion masiva quedaron  4123 registros y el id del ultimo era 115704

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state,partner_id,price_unit
1940,115704,"[107050, Draft Invoice (* 107050)]",0.0,True,0.0,False,False,"[48810, 5be3127ae4b0e9b0e27b3220]",2.53
1939,115703,"[107050, Draft Invoice (* 107050)]",0.0,True,0.0,False,False,"[48810, 5be3127ae4b0e9b0e27b3220]",2.53
1938,115702,"[107050, Draft Invoice (* 107050)]",0.0,True,0.0,False,False,"[48810, 5be3127ae4b0e9b0e27b3220]",2.53
1937,115701,"[107050, Draft Invoice (* 107050)]",0.0,True,0.0,False,False,"[48810, 5be3127ae4b0e9b0e27b3220]",2.53
1936,115700,"[107050, Draft Invoice (* 107050)]",0.0,True,0.0,False,False,"[48810, 5be3127ae4b0e9b0e27b3220]",2.53
...,...,...,...,...,...,...,...,...,...
3409,5,"[1, INV/2024/00002]",0.0,True,0.0,False,False,"[14, Azure Interior]",4000.00
4092,4,"[16, MISC/2024/07/0002]",0.0,True,0.0,False,False,"[10, Deco Addict]",0.00
4091,3,"[16, MISC/2024/07/0002]",0.0,True,0.0,False,False,"[10, Deco Addict]",0.00
4094,2,"[15, MISC/2024/07/0001]",0.0,True,0.0,False,False,"[10, Deco Addict]",0.00


In [324]:
sorted_last_invoices_lines = last_invoices_lines.sort_values(by=['move_id'], ascending=[False])
sorted_last_invoices_lines

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state,partner_id,price_unit
92,111675,"[105664, Draft Invoice (* 105664)]",0.0,True,0.0,False,False,"[48852, 664f96eab387d4068059b109]",39.0
89,111672,"[105664, Draft Invoice (* 105664)]",0.0,True,0.0,False,False,"[48852, 664f96eab387d4068059b109]",0.0
88,111671,"[105664, Draft Invoice (* 105664)]",0.0,True,0.0,False,False,"[48852, 664f96eab387d4068059b109]",10.0
91,111674,"[105664, Draft Invoice (* 105664)]",0.0,True,0.0,False,False,"[48852, 664f96eab387d4068059b109]",10.0
90,111673,"[105664, Draft Invoice (* 105664)]",0.0,True,0.0,False,False,"[48852, 664f96eab387d4068059b109]",10.0
...,...,...,...,...,...,...,...,...,...
1364,109723,"[105035, Draft Invoice (* 105035)]",0.0,True,0.0,False,False,"[49442, 5aa291c2e4b0c65fcc798ce8]",0.0
1363,109722,"[105035, Draft Invoice (* 105035)]",0.0,True,0.0,False,False,"[49442, 5aa291c2e4b0c65fcc798ce8]",30.0
1362,109721,"[105034, Draft Invoice (* 105034)]",0.0,True,0.0,False,False,"[49443, 5acdf7d8e4b014cba3b1724b]",89.0
1361,109720,"[105034, Draft Invoice (* 105034)]",0.0,True,0.0,False,False,"[49443, 5acdf7d8e4b014cba3b1724b]",0.0


In [325]:
unique_sorted_last_invoices_lines = sorted_last_invoices_lines.drop_duplicates('move_id')
unique_sorted_last_invoices_lines

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state,partner_id,price_unit
92,111675,"[105664, Draft Invoice (* 105664)]",0.0,True,0.0,False,False,"[48852, 664f96eab387d4068059b109]",39.00
85,111668,"[105663, Draft Invoice (* 105663)]",0.0,True,0.0,False,False,"[48853, 6424c2b0c5f9150025a4e2a8]",0.00
82,111665,"[105662, Draft Invoice (* 105662)]",0.0,True,0.0,False,False,"[48854, 5b58bda5e4b064b3a282db1d]",21.68
79,111662,"[105661, Draft Invoice (* 105661)]",0.0,True,0.0,False,False,"[48855, 5976056ae4b022062c263fd5]",0.00
76,111659,"[105660, Draft Invoice (* 105660)]",0.0,True,0.0,False,False,"[48856, 5b312739e4b06f222b92c69e]",100.64
...,...,...,...,...,...,...,...,...,...
1373,109732,"[105038, Draft Invoice (* 105038)]",0.0,True,0.0,False,False,"[49439, 5b47de04e4b03c859495a0f7]",0.00
1371,109730,"[105037, Draft Invoice (* 105037)]",0.0,True,0.0,False,False,"[49440, 5cb4d5d0e4b0bd6bed7869fa]",39.00
1368,109727,"[105036, Draft Invoice (* 105036)]",0.0,True,0.0,False,False,"[49441, 5ad08c66e4b01bf0eefaf887]",39.00
1365,109724,"[105035, Draft Invoice (* 105035)]",0.0,True,0.0,False,False,"[49442, 5aa291c2e4b0c65fcc798ce8]",89.00


In [327]:
#mas adelante intente hacer una eliminacion de muchos line_items pero resulta que al ingresar una nueva invoice, se tiene que eliminar 
# el asiento contable para que se pueda eliminar tanto la invoice como sus line_items
# asi que primero decido separar el campo move_id ya que es un one2many y necesito convertirlo a numerico para poderlo eliminar

# Separar la columna move_id en dos columnas: move_id (numérico) y description
last_invoices_lines[['move_id_num', 'description']] = pd.DataFrame(last_invoices_lines['move_id'].tolist(), index=last_invoices_lines.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices_lines['move_id_num'] = pd.to_numeric(last_invoices_lines['move_id_num'], errors='coerce')

# Verificar el resultado
last_invoices_lines_refined = last_invoices_lines[['id', 'move_id_num', 'description']]
last_invoices_lines_refined

,id,move_id_num,description
92,111675,105664,Draft Invoice (* 105664)
91,111674,105664,Draft Invoice (* 105664)
90,111673,105664,Draft Invoice (* 105664)
89,111672,105664,Draft Invoice (* 105664)
88,111671,105664,Draft Invoice (* 105664)
...,...,...,...
1364,109723,105035,Draft Invoice (* 105035)
1363,109722,105035,Draft Invoice (* 105035)
1362,109721,105034,Draft Invoice (* 105034)
1361,109720,105034,Draft Invoice (* 105034)


In [328]:
#last_moves que genere antes ahora lo uno con el dataframe que me permite separar el move_id de su numero y descripcion en el modelo
#account.move.line porque ese id lo muestra con un array tipo asi [45268, 'Draft Invoice (* 45268)']
lines_i_join = pd.merge(last_moves, last_invoices_lines_refined, on='move_id_num', how='inner')
lines_i_join

,move_id_num,partnerID,customer_id,id,description
0,105664,48852,664f96eab387d4068059b109,111675,Draft Invoice (* 105664)
1,105664,48852,664f96eab387d4068059b109,111674,Draft Invoice (* 105664)
2,105664,48852,664f96eab387d4068059b109,111673,Draft Invoice (* 105664)
3,105664,48852,664f96eab387d4068059b109,111672,Draft Invoice (* 105664)
4,105664,48852,664f96eab387d4068059b109,111671,Draft Invoice (* 105664)
...,...,...,...,...,...
1952,105035,49442,5aa291c2e4b0c65fcc798ce8,109723,Draft Invoice (* 105035)
1953,105035,49442,5aa291c2e4b0c65fcc798ce8,109722,Draft Invoice (* 105035)
1954,105034,49443,5acdf7d8e4b014cba3b1724b,109721,Draft Invoice (* 105034)
1955,105034,49443,5acdf7d8e4b014cba3b1724b,109720,Draft Invoice (* 105034)


In [350]:
#buscar un registro en account.move.line por medio de su id

#lines_i_join.loc[lines_i_join['move_id_num']==83370]

In [329]:
# Reversar los movimientos contables
move_ids = last_invoices_lines['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN
print("IDs de movimientos a procesar:", move_ids)


IDs de movimientos a procesar: [105664, 105664, 105664, 105664, 105664, 105663, 105663, 105663, 105662, 105662, 105662, 105661, 105661, 105661, 105660, 105660, 105660, 105659, 105659, 105658, 105658, 105658, 105657, 105657, 105656, 105656, 105656, 105656, 105655, 105655, 105655, 105654, 105654, 105654, 105653, 105653, 105653, 105652, 105652, 105652, 105651, 105651, 105651, 105650, 105650, 105650, 105649, 105649, 105649, 105648, 105648, 105647, 105647, 105647, 105646, 105646, 105646, 105646, 105646, 105643, 105643, 105643, 105642, 105642, 105642, 105641, 105641, 105641, 105640, 105640, 105640, 105640, 105640, 105640, 105640, 105640, 105640, 105640, 105631, 105631, 105631, 105630, 105630, 105630, 105630, 105630, 105630, 105629, 105629, 105629, 105628, 105628, 105628, 105627, 105627, 105627, 105626, 105626, 105626, 105626, 105624, 105624, 105624, 105624, 105624, 105623, 105623, 105623, 105622, 105622, 105622, 105621, 105621, 105620, 105620, 105620, 105619, 105619, 105619, 105618, 105618, 

In [331]:
# Reversar los movimientos contables osea eliminar todos los registros contables, lo que consiste en eliminar los line items de cada
# invoice y a la vez tambien se elimine el registro padre que es el de account.move, pero esta manera de liminarlos fue la que pude
#utilizar :(
move_ids = last_invoices_lines_refined['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN

# Usar un conjunto para asegurarse de no procesar IDs duplicados
processed_ids = set()

if move_ids:
    try:
        for move_id in move_ids:
            # Verificar si el ID ya ha sido procesado
            if move_id in processed_ids:
                print(f"ID de movimiento {move_id} ya fue procesado, saltando...")
                continue
            
            print(f"Procesando ID de movimiento: {move_id} (Tipo: {type(move_id)})")
            
            # Verificar si el movimiento existe
            move_record = odoo.execute('account.move', 'search_read', [[('id', '=', move_id)]], {'fields': ['state']})
            
            if move_record:
                if move_record[0]['state'] != 'draft':
                    # Si el estado no es 'draft', cambiarlo a 'draft'
                    odoo.execute('account.move', 'button_draft', [[move_id]])
                    print(f"Asiento contable {move_id} puesto en estado borrador.")
                
                # Eliminar el movimiento contable
                odoo.execute('account.move', 'unlink', [[move_id]])
                print(f"Asiento contable {move_id} eliminado.")
                
                # Marcar el ID como procesado
                processed_ids.add(move_id)
            else:
                # Si no encuentra el movimiento, continuar con el siguiente
                print(f"Asiento contable {move_id} no encontrado, posiblemente ya ha sido eliminado previamente.")
                processed_ids.add(move_id)  # Marcarlo como procesado para evitar reintentos
                continue  # Sigue con el siguiente movimiento

        print(f"Se han revertido y eliminado {len(processed_ids)} asientos contables.")
        
    except Exception as e:
        print(f"Error al revertir/eliminar los asientos: {str(e)}")
else:
    print("No se encontraron IDs válidos para revertir/eliminar.")


Procesando ID de movimiento: 105664 (Tipo: <class 'int'>)
Asiento contable 105664 eliminado.
ID de movimiento 105664 ya fue procesado, saltando...
ID de movimiento 105664 ya fue procesado, saltando...
ID de movimiento 105664 ya fue procesado, saltando...
ID de movimiento 105664 ya fue procesado, saltando...
Procesando ID de movimiento: 105663 (Tipo: <class 'int'>)
Asiento contable 105663 eliminado.
ID de movimiento 105663 ya fue procesado, saltando...
ID de movimiento 105663 ya fue procesado, saltando...
Procesando ID de movimiento: 105662 (Tipo: <class 'int'>)
Asiento contable 105662 eliminado.
ID de movimiento 105662 ya fue procesado, saltando...
ID de movimiento 105662 ya fue procesado, saltando...
Procesando ID de movimiento: 105661 (Tipo: <class 'int'>)
Asiento contable 105661 eliminado.
ID de movimiento 105661 ya fue procesado, saltando...
ID de movimiento 105661 ya fue procesado, saltando...
Procesando ID de movimiento: 105660 (Tipo: <class 'int'>)
Asiento contable 105660 elimin

In [ ]:
#habian 94 registros

In [107]:
#cbli.loc[(cbli['cn_id'].notna()) & (cbli['cn_id'].notna())]

In [112]:
#de aqui en adelante pode signorar el codigo ya que fueron pruebas de inserciones posibles en los otros modelos, tratando de insertar los datos de chargebee correspondientes 
#para tenerlos adaptados al modelo de odoo que esta muy normalizado en terminos de base de datos

,id_line_items_expanded,date_from,date_to,unit_amount,quantity,amount,pricing_model,is_taxed,tax_amount,object_line_items_expanded,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
22,li_Azyk9vUKCbLnIEIQ,2024-08-01,2024-09-01 09:39:00-03:00,0,1,0,tiered,False,0,line_item,...,NaN,line_item_discount,li_169mLZUKCkROfJfI,item_level_coupon,4340.0,2024[ESCORPO]10%FOREVER,2024[ESCORPO]10%FOREVER,NaN,NaN,NaN
111,li_16BdHGUKDFsgM2jJ6,2024-08-01,2024-09-01 12:20:00-03:00,0,1,0,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,li_16BdHGUKDZlfT3m2p,2024-08-01,2024-09-01 13:39:00-03:00,33,400,13000,stairstep,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,customer
156,li_16BdHGUKDZlfY3m2q,2024-08-01,2024-09-01 13:39:00-03:00,0,1,0,flat_fee,False,0,line_item,...,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,li_16BdHGUKDZlfZ3m2r,2024-08-01,2024-09-01 13:39:00-03:00,0,6,0,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,li_16BdGRULFHo7lMc4C,2024-08-12,2024-09-12 11:09:05-03:00,3900,1,3900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2297,li_16BdGRULGVQFvPNwC,2024-08-12,2024-09-12 16:09:26-03:00,0,1,0,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2307,li_Azz6neULGd4In5LSi,2024-08-12,2024-09-12 16:39:48-03:00,9900,1,9900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2395,li_16BdPRULLKygg4YsZ,2024-08-13,2024-09-13 11:59:18-03:00,0,3,0,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
i10 = odoo.search_read('account.move.reversal', [], [], limit=None)

# Convertir los resultados a un DataFrame
df_credit_notes = pd.DataFrame(i10)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_li_cn = df_credit_notes.sort_values(by='id', ascending=False).head(27)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_credit_notes = pd.DataFrame(last_li_cn)
last_credit_notes

,id,move_ids,new_move_ids,date,reason,journal_id,company_id,available_journal_ids,country_code,residual,...,l10n_latam_use_documents,l10n_latam_document_type_id,l10n_latam_available_document_type_ids,l10n_latam_document_number,l10n_latam_manual_document_number,l10n_es_edi_facturae_reason_code,l10n_cl_edi_reference_doc_code,l10n_cl_is_text_correction,l10n_cl_original_text,l10n_cl_corrected_text
5,6,[],[],2024-08-01,demo_sup_refund_invoice_5: liquido producto bi...,"[42, Vendor Bills]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[142, (186) DIRECT PURCHASE LIQUIDATION A - LI...",[],00011-00000012,False,10,False,False,False,False
4,5,[],[],2024-08-15,Venta cancelada,"[42, Vendor Bills]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[119, FOREIGN CREDIT NOTES AND REIMBURSEMENTS]",[],0001-01234566,False,10,False,False,False,False
3,4,[],[],2024-08-15,Mercadería defectuosa,"[42, Vendor Bills]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[119, FOREIGN CREDIT NOTES AND REIMBURSEMENTS]",[],0001-01234567,False,10,False,False,False,False
2,3,[],[],2024-08-01,Venta cancelada,"[51, Expo Sales Journal]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[19, (21) CREDIT NOTES FOR FOREIGN OPERATIONS]",[],False,False,10,False,False,False,False
1,2,[],[],2024-08-01,Venta cancelada,"[41, Ventas Preimpreso]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[3, (3) CREDIT NOTES A]",[],False,False,10,False,False,False,False
0,1,[],[],2024-08-01,Mercadería defectuosa,"[41, Ventas Preimpreso]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[3, (3) CREDIT NOTES A]",[],False,False,10,False,False,False,False


In [89]:
# Simulación de los impuestos disponibles en Odoo (ejemplo)


def create_move(odoo):
    """
    Esta función simula la creación de un movimiento contable (account.move).
    """
    move_vals = {
        'journal_id': 1,  # ID del diario contable
        'date': '2024-10-10',  # Fecha del movimiento
        'ref': 'Referencia de prueba',  # Referencia del movimiento
    }
    move_id = odoo.create('account.move', [move_vals])  # Crear un nuevo account.move
    return move_id

def get_aml_vals(product_id, partner_id, price, quantity):
    """
    Función que retorna los valores de una línea de 'account.move.line'.
    """
    return {
        'product_id': product_id,  # ID del producto
        'partner_id': partner_id,  # ID del cliente
        'name': 'Venta de producto',  # Descripción
        'quantity': quantity,  # Cantidad
        'price_unit': price,  # Precio unitario
        'debit': 0.0,  # Débito (en caso de ser necesario)
        'credit': price * quantity,  # Crédito
        'currency_id': 1,  # ID de la moneda # Impuesto aplicado
    }

def insert_lines_in_batch(odoo, products, partner_id):
    """
    Función que inserta líneas de account.move.line en batch para un account.move.
    
    - products: Lista de productos con su información.
    - partner_id: ID del cliente/partner para asociar a las líneas.
    """
    # 1. Crear el movimiento contable (account.move)
    move_id = create_move(odoo)

    # 2. Preparar las líneas de account.move.line
    aml_vals = []
    
    for product in products:
        product_id = product['product_id']
        price = product['price']
        quantity = product['quantity']
        
        # Generar los valores de la línea
        aml_vals.append(get_aml_vals(product_id, partner_id, price, quantity))
    
    # 3. Insertar las líneas de account.move.line usando create
    try:
        for aml in aml_vals:
            aml['move_id'] = move_id  # Asignar el ID del movimiento contable
            odoo.create('account.move.line', [aml])  # Crear las líneas individualmente
        print(f"Líneas insertadas correctamente en el movimiento con ID {move_id}.")
    except Exception as e:
        print("Error al insertar líneas:", str(e))

# Ejemplo de uso:
products = [
    {'product_id': 101, 'price': 100.0, 'quantity': 1},  # Producto 1
    {'product_id': 102, 'price': 200.0, 'quantity': 2},  # Producto 2
]

partner_id = 34827  # ID del cliente

# Insertar en batch
insert_lines_in_batch(odoo, products, partner_id)


{'jsonrpc': '2.0', 'id': 60, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.UserError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in route_

In [84]:
def insert_lines_in_batch(odoo, products, partner_id):
    """
    Función que inserta líneas de account.move.line en batch para un account.move.
    
    - products: Lista de productos con su información.
    - partner_id: ID del cliente/partner para asociar a las líneas.
    """
    # 1. Crear el movimiento contable (account.move)
    move_id = create_move(odoo)

    # 2. Preparar las líneas de account.move.line
    aml_vals = []
    
    total_debit = 0  # Acumular total de débitos
    for product in products:
        product_id = product['product_id']
        price = product['price']
        quantity = product['quantity']
        
        # Generar los valores de la línea sin impuestos
        aml_vals.append(get_aml_vals(product_id, partner_id, price, quantity))
        total_debit += price * quantity  # Sumar al total de débitos

    # Agregar una línea de crédito para equilibrar
    aml_vals.append({
        'product_id': False,  # Sin producto
        'partner_id': partner_id,  # ID del cliente
        'name': 'Contrapartida',  # Descripción
        'quantity': 1,  # Cantidad de 1 para el crédito
        'price_unit': total_debit,  # Usar el total de débitos como precio unitario
        'debit': 0.0,  # Sin débito
        'credit': total_debit,  # Total de créditos
        'currency_id': 1,  # ID de la moneda
    })

    # Verificar si los totales de débitos y créditos están balanceados
    total_credit = sum(aml['credit'] for aml in aml_vals)
    if total_debit != total_credit:
        print("Error: Los totales de débitos y créditos no están balanceados.")
        return

    # 3. Insertar las líneas de account.move.line usando create
    try:
        for aml in aml_vals:
            aml['move_id'] = move_id  # Asignar el ID del movimiento contable
            odoo.create('account.move.line', [aml])  # Crear las líneas individualmente
        print(f"Líneas insertadas correctamente en el movimiento con ID {move_id}.")
    except Exception as e:
        print("Error al insertar líneas:", str(e))

# Ejemplo de uso:
products = [
    {'product_id': 101, 'price': 100.0, 'quantity': 1},  # Producto 1
    {'product_id': 102, 'price': 200.0, 'quantity': 2},  # Producto 2
]

partner_id = 34827  # ID del cliente

# Insertar en batch
insert_lines_in_batch(odoo, products, partner_id)


{'jsonrpc': '2.0', 'id': 54, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.UserError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in route_

In [75]:
# Búsqueda de productos en el modelo 'product.product'
product_search = odoo.search_read('product.product', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
products_df = pd.DataFrame(product_search)

# Mostrar los productos encontrados
products_df

,id,name
0,94,Ad-Valorem
1,88,Tariff
2,86,Import Clearance
3,89,Guard Service
4,87,Statistics Rate
...,...,...
104,112,Senior Architect (Invoice on Timesheets)
105,110,Service on Timesheets
106,37,Top-up eWallet
107,4,Virtual Home Staging


In [153]:
products_df.loc[products_df['id'] == 118]

,id,name
73,118,Producto de ejemplo


In [70]:
# Datos del nuevo producto a insertar
new_product_data = {
    'name': 'Producto de ejemplo',
    'type': 'consu',  # tipo de producto ('consu' = Consumible, 'service' = Servicio, 'product' = Almacenable)
    'list_price': 100.00,  # Precio de venta
    'standard_price': 70.00,  # Precio de coste
    'default_code': 'PROD_EX',  # Código interno del producto
    'sale_ok': True,  # Habilitar para ventas
    'purchase_ok': True  # Habilitar para compras
}

# Insertar el nuevo producto usando el método Odoo.create
try:
    new_product_id = odoo.create('product.product', [new_product_data])
    print(f"Producto insertado con éxito. ID: {new_product_id}")
except Exception as e:
    print(f"Error durante la inserción del producto: {str(e)}")


Producto insertado con éxito. ID: 118


In [120]:
# Definir el ID de producto a buscar
product_id_to_delete = 118

# Búsqueda de productos en el modelo 'stock.move'
inventory = odoo.search_read('stock.move', [], ['product_id'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
inventory_df = pd.DataFrame(inventory)

# Extraer el ID del campo 'product_id', que es una lista [ID, Nombre]
# Crear una nueva columna en el DataFrame que contenga solo el número de product_id
inventory_df['product_id_number'] = inventory_df['product_id'].apply(lambda x: x[0] if isinstance(x, list) else None)

# Buscar el registro que tenga el product_id igual a 118
result = inventory_df.loc[inventory_df['product_id_number'] == product_id_to_delete]

# Mostrar los resultados encontrados
if not result.empty:
    print(f"Se encontró el producto con ID {product_id_to_delete} en 'el modelo':\n")
    print(result)
else:
    print(f"No se encontró el producto con ID {product_id_to_delete} en 'el modelo'.")


No se encontró el producto con ID 118 en 'el modelo'.


In [66]:
#hacer el llamado a res.users para mas adelante poder insertar los line items
# Especificar los campos que deseas leer
fields_to_read = ['id', 'name']

# Realizar el llamado al modelo account.move.line con un límite de registros
users = odoo.search_read('res.users', [], fields_to_read, limit=1000)

# Convertir los resultados a un DataFrame
u_df = pd.DataFrame(users)
u_df

,id,name
0,15,Chargebee
1,16,Jim Galvez
2,7,Joel Willis
3,6,Marc Demo
4,10,Valentino
5,11,Valentino
6,14,Valentino
7,13,Valentino
8,12,Valentino


In [119]:
#para no estorbar tanto en las lineas de arriba, hasta qui abajo genere como los queries para los dataframes en los modelos utilizados

#order.select_dtypes(include=['int64', 'float64'])

In [120]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
order.select_dtypes(include=['int64'])


,net_term_days,total,amount_paid,amount_adjusted,write_off_amount,credits_applied,amount_due,resource_version,amount_to_collect,round_off_amount,tax,sub_total,unit_amount,quantity,amount,tax_amount,discount_amount,item_level_discount_amount
0,0,8900,8900,0,0,0,0,1724435719786,0,0,0,8900,8900,1,8900,0,0,0
1,0,9900,9900,0,0,0,0,1723568155857,0,0,0,9900,0,1,0,0,0,0
2,0,3900,3900,0,0,0,0,1722508742063,0,0,0,3900,8900,1,8900,0,0,0
3,0,3900,3900,0,0,0,0,1723218079989,0,0,0,3900,750,4,3000,0,2000,2000
4,0,23900,23900,0,0,0,0,1722510311847,0,0,0,23900,39,100,3900,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,0,0,0,0,0,0,0,1725159618918,0,0,0,0,3000,1,3000,0,0,0
2451,0,0,0,0,0,0,0,1725159619104,0,0,0,0,48,50,2400,0,0,0
2452,0,0,0,0,0,0,0,1725159619316,0,0,0,0,0,3,0,0,0,0
2444,0,0,0,0,0,0,0,1725159635973,0,0,0,0,467,3,1400,0,1400,1400


In [316]:
customerorder1 = order.loc[order['customer_id'] == '6538aee581e02900329122e6']
customerorder1[['id','customer_id', 'total']]

,id,customer_id,total
155,185552,6538aee581e02900329122e6,16466
275,185675,6538aee581e02900329122e6,686100
1223,186658,6538aee581e02900329122e6,606
1224,186659,6538aee581e02900329122e6,606
1226,186661,6538aee581e02900329122e6,605
1227,186662,6538aee581e02900329122e6,605
1228,186663,6538aee581e02900329122e6,605
1229,186664,6538aee581e02900329122e6,605
1230,186665,6538aee581e02900329122e6,605
1232,186667,6538aee581e02900329122e6,604


KeyError: 'total'

In [296]:
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
string,Analytic Distribution,Analytic Distribution Search,Analytic Precision,Journal Entry,Journal,Company,Company Currency,Number,Status,Date,...,Origin,Payment Matched Amount,Group,Filter Amount,Tax Settlement Move,Tax State,Subscription,Monthly Recurring Revenue,Related Stock Moves,Withholding
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
oo = order.loc[order['customer_id'] == '6538aee581e02900329122e6']
oo

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
155,185552,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-01 17:19:07-03:00,2024-08-01 17:19:07-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
275,185675,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-02 17:33:55-03:00,2024-08-02 17:33:55-03:00,0,...,NaN,line_item_discount,li_16BdPgUKJz9CJ6lkn,item_level_coupon,19000.0,LEADAKI100%,LEADAKI100%,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1222,186658,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:55:09-03:00,2024-08-14 22:55:09-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1223,186659,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:59:53-03:00,2024-08-14 22:59:53-03:00,0,...,NaN,line_item_discount,li_Azz6ewULS8wFn1SYu,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1225,186661,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:07:08-03:00,2024-08-14 23:07:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1226,186662,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:11:58-03:00,2024-08-14 23:11:58-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1227,186663,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:20:18-03:00,2024-08-14 23:20:18-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1228,186664,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:28:43-03:00,2024-08-14 23:28:43-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1229,186665,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:45:33-03:00,2024-08-14 23:45:33-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1231,186667,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-15 00:09:58-03:00,2024-08-15 00:09:58-03:00,0,...,NaN,line_item_discount,li_16BdC4ULSSprV83UH,item_level_coupon,700.0,MIGRA24A23STARTER1AGENTEX3MESES,MIGRA24A23STARTER1AGENTEX3MESES,NaN,NaN,NaN


In [151]:
oo[['customer_id', 'amount', 'quantity', 'total']]

,customer_id,amount,quantity,total
155,6538aee581e02900329122e6,13000,400,16466
275,6538aee581e02900329122e6,0,6,686100
1222,6538aee581e02900329122e6,13000,400,606
1223,6538aee581e02900329122e6,2000,1,606
1225,6538aee581e02900329122e6,700,2,605
1226,6538aee581e02900329122e6,3900,1,605
1227,6538aee581e02900329122e6,0,1,605
1228,6538aee581e02900329122e6,2400,50,605
1229,6538aee581e02900329122e6,0,3,605
1231,6538aee581e02900329122e6,3000,4,604


In [152]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
order_floats = order.select_dtypes(include=['float64'])
order_floats

,exchange_rate,new_sales_amount,expected_payment_date,next_retry_at,starting_unit,ending_unit,quantity_used,unit_amount_line_item_tiers_expanded,applied_amount,txn_date,...,cn_date_adjustment_credit_notes_expanded,cn_total,cn_date_issued_credit_notes_expanded,cn_total_issued_credit_notes_expanded,created_at,attempt,txn_amount_dunning_attempts_expanded,amount_discounts_expanded,discount_percentage,discount_amount_line_item_discounts_expanded
0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,8900.0,1.724349e+09,...,NaN,NaN,NaN,NaN,1.722568e+09,1.0,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,9900.0,1.723050e+09,...,NaN,NaN,NaN,NaN,1.723518e+09,2.0,NaN,2000.0,NaN,2000.0
2,1.0,NaN,NaN,NaN,2.0,NaN,3.0,1000.0,3900.0,1.721012e+09,...,NaN,NaN,NaN,NaN,1.724382e+09,3.0,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,1.0,3.0,3.0,0.0,3900.0,1.723045e+09,...,NaN,NaN,NaN,NaN,1.722568e+09,1.0,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,23900.0,1.722510e+09,...,NaN,NaN,NaN,NaN,1.723518e+09,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,9000.0,1.724288e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8900.0,100.0,NaN
2454,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,3900.0,1.724282e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9900.0,100.0,NaN
2455,1.0,NaN,NaN,NaN,2.0,NaN,1.0,700.0,3900.0,1.724283e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3900.0,100.0,NaN
2447,1.0,NaN,NaN,NaN,1.0,6.0,6.0,0.0,3900.0,1.724276e+09,...,NaN,NaN,NaN,NaN,1.724345e+09,0.0,3400.0,7400.0,100.0,NaN


In [318]:
ono = lines_join_clean.loc[lines_join_clean['customer_id'] == '6538aee581e02900329122e6']
ono

,invoice_id,PartnerID,customer_id,id,po_number,subscription_id,recurring,status,price_type,date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
61,95371,44756,6538aee581e02900329122e6,185552,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-01 17:19:07-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
62,95371,44756,6538aee581e02900329122e6,185675,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-02 17:33:55-03:00,...,NaN,line_item_discount,li_16BdPgUKJz9CJ6lkn,item_level_coupon,19000.0,LEADAKI100%,LEADAKI100%,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
63,95371,44756,6538aee581e02900329122e6,186658,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:55:09-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,95371,44756,6538aee581e02900329122e6,186659,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:59:53-03:00,...,NaN,line_item_discount,li_Azz6ewULS8wFn1SYu,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
65,95371,44756,6538aee581e02900329122e6,186661,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:07:08-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,95371,44756,6538aee581e02900329122e6,186662,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:11:58-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
67,95371,44756,6538aee581e02900329122e6,186663,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:20:18-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
68,95371,44756,6538aee581e02900329122e6,186664,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:28:43-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
69,95371,44756,6538aee581e02900329122e6,186665,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:45:33-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
70,95371,44756,6538aee581e02900329122e6,186667,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-15 00:09:58-03:00,...,NaN,line_item_discount,li_16BdC4ULSSprV83UH,item_level_coupon,700.0,MIGRA24A23STARTER1AGENTEX3MESES,MIGRA24A23STARTER1AGENTEX3MESES,NaN,NaN,NaN


In [330]:
ono[['total', 'quantity', 'amount']]

,total,quantity,amount
61,16466,400,13000
62,686100,6,0
63,606,2,1000
64,606,400,13000
65,605,1,3900
66,605,2,700
67,605,1,3900
68,605,1,0
69,605,50,2400
70,604,1,8900
